In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.tree import _tree
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np
import re
import random
import copy
import json

In [2]:
# Script Outline
# Input: row
# Output: paths(num >=1)
# For each entry, there should be (path, prediction, accuracy, the missing features)
# Path shoud have a dict of each values 
# Note: There should be a measture for the missing features
#       Broken Path(features all missing) should be handled

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
pd.set_option('display.max_rows', None)  # None means unlimited
pd.set_option('display.max_columns', None)  # None means unlimited
pd.set_option('display.width', 1000)  # Adjust width to accommodate more columns

In [5]:
random.seed(42)

In [6]:
numerical_features = [
    'flow_duration', 'Header_Length', 'Duration', 'Rate',
    'Srate', 'Drate', 'ack_count','syn_count','fin_count','urg_count',
    'rst_count','Tot_sum', 'Min', 'Max', 'AVG', 'Std',
    'Tot_size', 'IAT', 'Number', 'Magnitue', 'Radius',
    'Covariance', 'Variance', 'Weight'
]

flag_features = [
    'HTTP', 'HTTPS', 'DNS', 'Telnet',
    'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP',
    'ICMP', 'IPv', 'LLC'
]

all_features = [
    'flow_duration', 'Header_Length', 'Duration', 'Rate',
    'Srate', 'Drate', 'ack_count','syn_count','fin_count','urg_count',
    'rst_count','Tot_sum', 'Min', 'Max', 'AVG', 'Std',
    'Tot_size', 'IAT', 'Number', 'Magnitue', 'Radius',
    'Covariance', 'Variance', 'Weight',
    'HTTP', 'HTTPS', 'DNS', 'Telnet',
    'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP',
    'ICMP', 'IPv', 'LLC',
    'fin_flag_number', 'syn_flag_number', 'rst_flag_number',
    'psh_flag_number', 'ack_flag_number', 'ece_flag_number', 'cwr_flag_number'
]
#4 6 6 5 3 4 7 3 3 4

In [7]:
df = pd.read_csv('source/test.csv')
# df = df.groupby('label').sample(n=625, random_state=42)
df.rename(columns={'Tot sum': 'Tot_sum'}, inplace=True)
df.rename(columns={'Tot size': 'Tot_size'}, inplace=True)
df.rename(columns={'Protocol Type': 'Protocol_Type'}, inplace=True)

In [8]:
all_entries_res = []

In [9]:
def classify(data):
    global all_entries_res
    potential_paths = []
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] > 2.58) or (data['Rate'] == -1)) and ((data['Tot_size'] <= 2061.0) or (data['Tot_size'] == -1)) and ((data['Rate'] <= 250.37) or (data['Rate'] == -1)) and ((data['HTTPS'] > 0.5) or (data['HTTPS'] == -1)) and ((data['flow_duration'] <= 136.71) or (data['flow_duration'] == -1)) and ((data['Variance'] <= 0.97) or (data['Variance'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'Tot_size': data['Tot_size'], 'HTTPS': data['HTTPS'], 'flow_duration': data['flow_duration'], 'Variance': data['Variance']}
        potential_paths.append([feature_dict, 'Benign', 91.77])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] <= 20.68) or (data['Std'] == -1)) and ((data['Header_Length'] <= 51990.2) or (data['Header_Length'] == -1)) and ((data['urg_count'] <= 2.7) or (data['urg_count'] == -1)) and ((data['Covariance'] <= 65.16) or (data['Covariance'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'Header_Length': data['Header_Length'], 'Covariance': data['Covariance']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] > 2.58) or (data['Rate'] == -1)) and ((data['Tot_size'] <= 2061.0) or (data['Tot_size'] == -1)) and ((data['Rate'] <= 250.37) or (data['Rate'] == -1)) and ((data['HTTPS'] > 0.5) or (data['HTTPS'] == -1)) and ((data['flow_duration'] <= 136.71) or (data['flow_duration'] == -1)) and ((data['Variance'] > 0.97) or (data['Variance'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'Tot_size': data['Tot_size'], 'HTTPS': data['HTTPS'], 'flow_duration': data['flow_duration'], 'Variance': data['Variance']}
        potential_paths.append([feature_dict, 'Benign', 98.04])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] > 166551400.0) or (data['IAT'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] > 2.58) or (data['Rate'] == -1)) and ((data['Tot_size'] <= 2061.0) or (data['Tot_size'] == -1)) and ((data['Rate'] <= 250.37) or (data['Rate'] == -1)) and ((data['HTTPS'] <= 0.5) or (data['HTTPS'] == -1)) and ((data['IAT'] <= 166499168.0) or (data['IAT'] == -1)) and ((data['Rate'] <= 82.11) or (data['Rate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'Tot_size': data['Tot_size'], 'HTTPS': data['HTTPS']}
        potential_paths.append([feature_dict, 'Benign', 74.38])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] > 2.58) or (data['Rate'] == -1)) and ((data['Tot_size'] <= 2061.0) or (data['Tot_size'] == -1)) and ((data['Rate'] <= 250.37) or (data['Rate'] == -1)) and ((data['HTTPS'] <= 0.5) or (data['HTTPS'] == -1)) and ((data['IAT'] > 166499168.0) or (data['IAT'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'Tot_size': data['Tot_size'], 'HTTPS': data['HTTPS']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 20.15) or (data['Magnitue'] == -1)) and ((data['Srate'] <= 63.78) or (data['Srate'] == -1)) and ((data['flow_duration'] <= 198.58) or (data['flow_duration'] == -1)) and ((data['Srate'] > 5.2) or (data['Srate'] == -1)) and ((data['IAT'] <= 166551312.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 521.15) or (data['Header_Length'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Srate': data['Srate'], 'flow_duration': data['flow_duration'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length']}
        potential_paths.append([feature_dict, 'Benign', 61.47])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] <= 20.68) or (data['Std'] == -1)) and ((data['Header_Length'] > 51990.2) or (data['Header_Length'] == -1)) and ((data['Header_Length'] > 82806.7) or (data['Header_Length'] == -1)) and ((data['flow_duration'] <= 57.88) or (data['flow_duration'] == -1)) and ((data['Tot_size'] > 99.6) or (data['Tot_size'] == -1)) and ((data['Magnitue'] <= 54.84) or (data['Magnitue'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'Header_Length': data['Header_Length'], 'flow_duration': data['flow_duration'], 'Tot_size': data['Tot_size'], 'Magnitue': data['Magnitue']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] <= 20.68) or (data['Std'] == -1)) and ((data['Header_Length'] <= 51990.2) or (data['Header_Length'] == -1)) and ((data['urg_count'] <= 2.7) or (data['urg_count'] == -1)) and ((data['Covariance'] > 65.16) or (data['Covariance'] == -1)) and ((data['Protocol_Type'] > 15.18) or (data['Protocol_Type'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'Header_Length': data['Header_Length'], 'Covariance': data['Covariance'], 'Protocol_Type': data['Protocol_Type']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] > 20.68) or (data['Std'] == -1)) and ((data['ack_flag_number'] <= 0.5) or (data['ack_flag_number'] == -1)) and ((data['urg_count'] <= 0.17) or (data['urg_count'] == -1)) and ((data['Tot_size'] > 55.15) or (data['Tot_size'] == -1)) and ((data['IAT'] > 41463032.02) or (data['IAT'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'ack_flag_number': data['ack_flag_number'], 'Tot_size': data['Tot_size'], 'IAT': data['IAT']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] > 193.08) or (data['Weight'] == -1)) and ((data['fin_count'] <= 0.65) or (data['fin_count'] == -1)) and ((data['AVG'] > 137.6) or (data['AVG'] == -1)) and ((data['IAT'] > 166499176.0) or (data['IAT'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'fin_count': data['fin_count'], 'AVG': data['AVG']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] > 2.58) or (data['Rate'] == -1)) and ((data['Tot_size'] <= 2061.0) or (data['Tot_size'] == -1)) and ((data['Rate'] > 250.37) or (data['Rate'] == -1)) and ((data['Number'] > 11.5) or (data['Number'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'Tot_size': data['Tot_size'], 'Number': data['Number']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] > 193.08) or (data['Weight'] == -1)) and ((data['fin_count'] <= 0.65) or (data['fin_count'] == -1)) and ((data['AVG'] <= 137.6) or (data['AVG'] == -1)) and ((data['Rate'] > 0.21) or (data['Rate'] == -1)) and ((data['Max'] > 224.2) or (data['Max'] == -1)) and ((data['Tot_size'] > 85.95) or (data['Tot_size'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'fin_count': data['fin_count'], 'AVG': data['AVG'], 'Rate': data['Rate'], 'Max': data['Max'], 'Tot_size': data['Tot_size']}
        potential_paths.append([feature_dict, 'Benign', 70.95])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] <= 193.08) or (data['Weight'] == -1)) and ((data['IAT'] <= 0.02) or (data['IAT'] == -1)) and ((data['SSH'] <= 0.5) or (data['SSH'] == -1)) and ((data['rst_count'] > 77.55) or (data['rst_count'] == -1)) and ((data['Protocol_Type'] > 6.45) or (data['Protocol_Type'] == -1)) and ((data['Header_Length'] > 22829.0) or (data['Header_Length'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'SSH': data['SSH'], 'rst_count': data['rst_count'], 'Protocol_Type': data['Protocol_Type']}
        potential_paths.append([feature_dict, 'Benign', 77.7])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] > 2.58) or (data['Rate'] == -1)) and ((data['Tot_size'] <= 2061.0) or (data['Tot_size'] == -1)) and ((data['Rate'] > 250.37) or (data['Rate'] == -1)) and ((data['Number'] <= 11.5) or (data['Number'] == -1)) and ((data['rst_count'] <= 2541.7) or (data['rst_count'] == -1)) and ((data['Srate'] > 2093.32) or (data['Srate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'Tot_size': data['Tot_size'], 'Number': data['Number'], 'rst_count': data['rst_count'], 'Srate': data['Srate']}
        potential_paths.append([feature_dict, 'Benign', 81.75])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] <= 436.1) or (data['Tot_sum'] == -1)) and ((data['fin_count'] > 0.05) or (data['fin_count'] == -1)) and ((data['Protocol_Type'] <= 10.65) or (data['Protocol_Type'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'fin_count': data['fin_count'], 'Protocol_Type': data['Protocol_Type']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 20.15) or (data['Magnitue'] == -1)) and ((data['Covariance'] <= 334993.23) or (data['Covariance'] == -1)) and ((data['Srate'] > 51.52) or (data['Srate'] == -1)) and ((data['Radius'] <= 514.89) or (data['Radius'] == -1)) and ((data['ack_flag_number'] <= 0.5) or (data['ack_flag_number'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Covariance': data['Covariance'], 'Srate': data['Srate'], 'Radius': data['Radius'], 'ack_flag_number': data['ack_flag_number']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 20.15) or (data['Magnitue'] == -1)) and ((data['Srate'] <= 63.78) or (data['Srate'] == -1)) and ((data['flow_duration'] <= 198.58) or (data['flow_duration'] == -1)) and ((data['Srate'] <= 5.2) or (data['Srate'] == -1)) and ((data['Duration'] > 60.9) or (data['Duration'] == -1)) and ((data['rst_count'] > 0.05) or (data['rst_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Srate': data['Srate'], 'flow_duration': data['flow_duration'], 'Duration': data['Duration']}
        potential_paths.append([feature_dict, 'Threat', 86.54])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] > 2.58) or (data['Rate'] == -1)) and ((data['Tot_size'] <= 2061.0) or (data['Tot_size'] == -1)) and ((data['Rate'] > 250.37) or (data['Rate'] == -1)) and ((data['Number'] <= 11.5) or (data['Number'] == -1)) and ((data['rst_count'] <= 2541.7) or (data['rst_count'] == -1)) and ((data['Srate'] <= 2093.32) or (data['Srate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'Tot_size': data['Tot_size'], 'Number': data['Number'], 'rst_count': data['rst_count'], 'Srate': data['Srate']}
        potential_paths.append([feature_dict, 'Threat', 50.49])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] > 2.58) or (data['Rate'] == -1)) and ((data['Tot_size'] > 2061.0) or (data['Tot_size'] == -1)) and ((data['Header_Length'] <= 4577017.75) or (data['Header_Length'] == -1)) and ((data['Variance'] > 0.95) or (data['Variance'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'Tot_size': data['Tot_size'], 'Variance': data['Variance']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] <= 193.08) or (data['Weight'] == -1)) and ((data['IAT'] <= 0.02) or (data['IAT'] == -1)) and ((data['SSH'] <= 0.5) or (data['SSH'] == -1)) and ((data['rst_count'] > 77.55) or (data['rst_count'] == -1)) and ((data['Protocol_Type'] > 6.45) or (data['Protocol_Type'] == -1)) and ((data['Header_Length'] <= 22829.0) or (data['Header_Length'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'SSH': data['SSH'], 'rst_count': data['rst_count'], 'Protocol_Type': data['Protocol_Type']}
        potential_paths.append([feature_dict, 'Benign', 58.7])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] <= 20.68) or (data['Std'] == -1)) and ((data['Header_Length'] <= 51990.2) or (data['Header_Length'] == -1)) and ((data['urg_count'] <= 2.7) or (data['urg_count'] == -1)) and ((data['Covariance'] > 65.16) or (data['Covariance'] == -1)) and ((data['Protocol_Type'] <= 15.18) or (data['Protocol_Type'] == -1)) and ((data['Header_Length'] <= 539.35) or (data['Header_Length'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'Header_Length': data['Header_Length'], 'Covariance': data['Covariance'], 'Protocol_Type': data['Protocol_Type']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] <= 193.08) or (data['Weight'] == -1)) and ((data['IAT'] <= 0.02) or (data['IAT'] == -1)) and ((data['SSH'] <= 0.5) or (data['SSH'] == -1)) and ((data['rst_count'] <= 77.55) or (data['rst_count'] == -1)) and ((data['Std'] <= 87.1) or (data['Std'] == -1)) and ((data['Radius'] <= 73.61) or (data['Radius'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'SSH': data['SSH'], 'rst_count': data['rst_count'], 'Std': data['Std'], 'Radius': data['Radius']}
        potential_paths.append([feature_dict, 'Threat', 60.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 20.15) or (data['Magnitue'] == -1)) and ((data['Covariance'] <= 334993.23) or (data['Covariance'] == -1)) and ((data['Srate'] <= 51.52) or (data['Srate'] == -1)) and ((data['rst_count'] <= 1.35) or (data['rst_count'] == -1)) and ((data['AVG'] > 230.36) or (data['AVG'] == -1)) and ((data['Covariance'] <= 100546.71) or (data['Covariance'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Covariance': data['Covariance'], 'Srate': data['Srate'], 'AVG': data['AVG']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 20.15) or (data['Magnitue'] == -1)) and ((data['Srate'] <= 63.78) or (data['Srate'] == -1)) and ((data['flow_duration'] <= 198.58) or (data['flow_duration'] == -1)) and ((data['Srate'] > 5.2) or (data['Srate'] == -1)) and ((data['IAT'] > 166551312.0) or (data['IAT'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Srate': data['Srate'], 'flow_duration': data['flow_duration'], 'IAT': data['IAT']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] > 2.58) or (data['Rate'] == -1)) and ((data['Tot_size'] > 2061.0) or (data['Tot_size'] == -1)) and ((data['Header_Length'] <= 4577017.75) or (data['Header_Length'] == -1)) and ((data['Variance'] <= 0.95) or (data['Variance'] == -1)) and ((data['flow_duration'] > 8.23) or (data['flow_duration'] == -1)) and ((data['Max'] > 2744.8) or (data['Max'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'Tot_size': data['Tot_size'], 'Variance': data['Variance'], 'flow_duration': data['flow_duration'], 'Max': data['Max']}
        potential_paths.append([feature_dict, 'Benign', 97.22])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] <= 193.08) or (data['Weight'] == -1)) and ((data['IAT'] > 0.02) or (data['IAT'] == -1)) and ((data['Tot_size'] > 119.09) or (data['Tot_size'] == -1)) and ((data['fin_count'] <= 0.0) or (data['fin_count'] == -1)) and ((data['Weight'] <= 90.03) or (data['Weight'] == -1)) and ((data['Srate'] <= 2.67) or (data['Srate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'Tot_size': data['Tot_size'], 'fin_count': data['fin_count'], 'Srate': data['Srate']}
        potential_paths.append([feature_dict, 'Threat', 84.06])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] <= 193.08) or (data['Weight'] == -1)) and ((data['IAT'] <= 0.02) or (data['IAT'] == -1)) and ((data['SSH'] <= 0.5) or (data['SSH'] == -1)) and ((data['rst_count'] <= 77.55) or (data['rst_count'] == -1)) and ((data['Std'] > 87.1) or (data['Std'] == -1)) and ((data['AVG'] > 223.55) or (data['AVG'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'SSH': data['SSH'], 'rst_count': data['rst_count'], 'Std': data['Std'], 'AVG': data['AVG']}
        potential_paths.append([feature_dict, 'Threat', 56.72])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 20.15) or (data['Magnitue'] == -1)) and ((data['Covariance'] <= 334993.23) or (data['Covariance'] == -1)) and ((data['Srate'] <= 51.52) or (data['Srate'] == -1)) and ((data['rst_count'] > 1.35) or (data['rst_count'] == -1)) and ((data['Rate'] <= 50.38) or (data['Rate'] == -1)) and ((data['Duration'] > 48.35) or (data['Duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Covariance': data['Covariance'], 'Srate': data['Srate'], 'Rate': data['Rate'], 'Duration': data['Duration']}
        potential_paths.append([feature_dict, 'Threat', 79.1])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] <= 193.08) or (data['Weight'] == -1)) and ((data['IAT'] > 0.02) or (data['IAT'] == -1)) and ((data['Tot_size'] <= 119.09) or (data['Tot_size'] == -1)) and ((data['Variance'] > 0.89) or (data['Variance'] == -1)) and ((data['flow_duration'] > 64.74) or (data['flow_duration'] == -1)) and ((data['Min'] <= 64.81) or (data['Min'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'Tot_size': data['Tot_size'], 'Variance': data['Variance'], 'flow_duration': data['flow_duration'], 'Min': data['Min']}
        potential_paths.append([feature_dict, 'Threat', 92.42])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] > 4.15) or (data['urg_count'] == -1)) and ((data['Max'] <= 1495.0) or (data['Max'] == -1)) and ((data['Srate'] > 32.65) or (data['Srate'] == -1)) and ((data['Header_Length'] <= 148793.2) or (data['Header_Length'] == -1)) and ((data['Header_Length'] > 30030.7) or (data['Header_Length'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Max': data['Max'], 'Srate': data['Srate'], 'Header_Length': data['Header_Length']}
        potential_paths.append([feature_dict, 'Threat', 88.33])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] > 193.08) or (data['Weight'] == -1)) and ((data['fin_count'] <= 0.65) or (data['fin_count'] == -1)) and ((data['AVG'] <= 137.6) or (data['AVG'] == -1)) and ((data['Rate'] > 0.21) or (data['Rate'] == -1)) and ((data['Max'] <= 224.2) or (data['Max'] == -1)) and ((data['Std'] > 24.99) or (data['Std'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'fin_count': data['fin_count'], 'AVG': data['AVG'], 'Rate': data['Rate'], 'Max': data['Max'], 'Std': data['Std']}
        potential_paths.append([feature_dict, 'Benign', 96.61])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 20.15) or (data['Magnitue'] == -1)) and ((data['Covariance'] > 334993.23) or (data['Covariance'] == -1)) and ((data['IAT'] <= 166551360.0) or (data['IAT'] == -1)) and ((data['Protocol_Type'] > 5.92) or (data['Protocol_Type'] == -1)) and ((data['flow_duration'] <= 81.93) or (data['flow_duration'] == -1)) and ((data['Protocol_Type'] <= 9.6) or (data['Protocol_Type'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Covariance': data['Covariance'], 'IAT': data['IAT'], 'Protocol_Type': data['Protocol_Type'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Benign', 61.11])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 20.15) or (data['Magnitue'] == -1)) and ((data['Srate'] > 63.78) or (data['Srate'] == -1)) and ((data['flow_duration'] <= 44.61) or (data['flow_duration'] == -1)) and ((data['syn_count'] > 0.75) or (data['syn_count'] == -1)) and ((data['TCP'] > 0.5) or (data['TCP'] == -1)) and ((data['Max'] > 179.8) or (data['Max'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Srate': data['Srate'], 'flow_duration': data['flow_duration'], 'syn_count': data['syn_count'], 'TCP': data['TCP'], 'Max': data['Max']}
        potential_paths.append([feature_dict, 'Threat', 96.08])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] <= 436.1) or (data['Tot_sum'] == -1)) and ((data['fin_count'] <= 0.05) or (data['fin_count'] == -1)) and ((data['syn_flag_number'] <= 0.5) or (data['syn_flag_number'] == -1)) and ((data['Tot_size'] > 57.55) or (data['Tot_size'] == -1)) and ((data['rst_count'] <= 14.15) or (data['rst_count'] == -1)) and ((data['syn_count'] <= 1.3) or (data['syn_count'] == -1)) and ((data['Tot_size'] <= 81.15) or (data['Tot_size'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'fin_count': data['fin_count'], 'syn_flag_number': data['syn_flag_number'], 'Tot_size': data['Tot_size'], 'rst_count': data['rst_count'], 'syn_count': data['syn_count']}
        potential_paths.append([feature_dict, 'Threat', 78.43])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] <= 193.08) or (data['Weight'] == -1)) and ((data['IAT'] > 0.02) or (data['IAT'] == -1)) and ((data['Tot_size'] > 119.09) or (data['Tot_size'] == -1)) and ((data['fin_count'] <= 0.0) or (data['fin_count'] == -1)) and ((data['Weight'] <= 90.03) or (data['Weight'] == -1)) and ((data['Srate'] > 2.67) or (data['Srate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'Tot_size': data['Tot_size'], 'fin_count': data['fin_count'], 'Srate': data['Srate']}
        potential_paths.append([feature_dict, 'Threat', 54.9])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] <= 193.08) or (data['Weight'] == -1)) and ((data['IAT'] <= 0.02) or (data['IAT'] == -1)) and ((data['SSH'] <= 0.5) or (data['SSH'] == -1)) and ((data['rst_count'] <= 77.55) or (data['rst_count'] == -1)) and ((data['Std'] <= 87.1) or (data['Std'] == -1)) and ((data['Radius'] > 73.61) or (data['Radius'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'SSH': data['SSH'], 'rst_count': data['rst_count'], 'Std': data['Std'], 'Radius': data['Radius']}
        potential_paths.append([feature_dict, 'Benign', 73.47])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 20.15) or (data['Magnitue'] == -1)) and ((data['Srate'] <= 63.78) or (data['Srate'] == -1)) and ((data['flow_duration'] > 198.58) or (data['flow_duration'] == -1)) and ((data['flow_duration'] <= 542.38) or (data['flow_duration'] == -1)) and ((data['Magnitue'] > 13.02) or (data['Magnitue'] == -1)) and ((data['Tot_sum'] > 619.15) or (data['Tot_sum'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Srate': data['Srate'], 'flow_duration': data['flow_duration'], 'Tot_sum': data['Tot_sum']}
        potential_paths.append([feature_dict, 'Threat', 93.62])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] > 4.15) or (data['urg_count'] == -1)) and ((data['Max'] > 1495.0) or (data['Max'] == -1)) and ((data['Tot_sum'] > 6064.9) or (data['Tot_sum'] == -1)) and ((data['Covariance'] <= 1139705.38) or (data['Covariance'] == -1)) and ((data['rst_count'] <= 474.5) or (data['rst_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Max': data['Max'], 'Tot_sum': data['Tot_sum'], 'Covariance': data['Covariance']}
        potential_paths.append([feature_dict, 'Benign', 54.35])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] <= 193.08) or (data['Weight'] == -1)) and ((data['IAT'] <= 0.02) or (data['IAT'] == -1)) and ((data['SSH'] <= 0.5) or (data['SSH'] == -1)) and ((data['rst_count'] > 77.55) or (data['rst_count'] == -1)) and ((data['Protocol_Type'] <= 6.45) or (data['Protocol_Type'] == -1)) and ((data['flow_duration'] <= 57.31) or (data['flow_duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'SSH': data['SSH'], 'rst_count': data['rst_count'], 'Protocol_Type': data['Protocol_Type'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Benign', 71.11])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] <= 4.15) or (data['urg_count'] == -1)) and ((data['Duration'] <= 127.4) or (data['Duration'] == -1)) and ((data['Tot_sum'] > 3211.7) or (data['Tot_sum'] == -1)) and ((data['IAT'] <= 166563728.0) or (data['IAT'] == -1)) and ((data['AVG'] <= 1508.07) or (data['AVG'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Duration': data['Duration'], 'Tot_sum': data['Tot_sum'], 'IAT': data['IAT'], 'AVG': data['AVG']}
        potential_paths.append([feature_dict, 'Benign', 81.4])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] > 2.58) or (data['Rate'] == -1)) and ((data['Tot_size'] > 2061.0) or (data['Tot_size'] == -1)) and ((data['Header_Length'] <= 4577017.75) or (data['Header_Length'] == -1)) and ((data['Variance'] <= 0.95) or (data['Variance'] == -1)) and ((data['flow_duration'] <= 8.23) or (data['flow_duration'] == -1)) and ((data['urg_count'] > 20.25) or (data['urg_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'Tot_size': data['Tot_size'], 'Variance': data['Variance'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Threat', 71.43])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] <= 193.08) or (data['Weight'] == -1)) and ((data['IAT'] <= 0.02) or (data['IAT'] == -1)) and ((data['SSH'] <= 0.5) or (data['SSH'] == -1)) and ((data['rst_count'] <= 77.55) or (data['rst_count'] == -1)) and ((data['Std'] > 87.1) or (data['Std'] == -1)) and ((data['AVG'] <= 223.55) or (data['AVG'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'SSH': data['SSH'], 'rst_count': data['rst_count'], 'Std': data['Std'], 'AVG': data['AVG']}
        potential_paths.append([feature_dict, 'Threat', 75.61])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] <= 193.08) or (data['Weight'] == -1)) and ((data['IAT'] > 0.02) or (data['IAT'] == -1)) and ((data['Tot_size'] <= 119.09) or (data['Tot_size'] == -1)) and ((data['Variance'] > 0.89) or (data['Variance'] == -1)) and ((data['flow_duration'] > 64.74) or (data['flow_duration'] == -1)) and ((data['Min'] > 64.81) or (data['Min'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'Tot_size': data['Tot_size'], 'Variance': data['Variance'], 'flow_duration': data['flow_duration'], 'Min': data['Min']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] > 2.58) or (data['Rate'] == -1)) and ((data['Tot_size'] <= 2061.0) or (data['Tot_size'] == -1)) and ((data['Rate'] > 250.37) or (data['Rate'] == -1)) and ((data['Number'] <= 11.5) or (data['Number'] == -1)) and ((data['rst_count'] > 2541.7) or (data['rst_count'] == -1)) and ((data['flow_duration'] <= 75.49) or (data['flow_duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'Tot_size': data['Tot_size'], 'Number': data['Number'], 'rst_count': data['rst_count'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Threat', 94.59])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 20.15) or (data['Magnitue'] == -1)) and ((data['Covariance'] > 334993.23) or (data['Covariance'] == -1)) and ((data['IAT'] > 166551360.0) or (data['IAT'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Covariance': data['Covariance'], 'IAT': data['IAT']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 20.15) or (data['Magnitue'] == -1)) and ((data['Srate'] > 63.78) or (data['Srate'] == -1)) and ((data['flow_duration'] > 44.61) or (data['flow_duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Srate': data['Srate'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 20.15) or (data['Magnitue'] == -1)) and ((data['Srate'] <= 63.78) or (data['Srate'] == -1)) and ((data['flow_duration'] > 198.58) or (data['flow_duration'] == -1)) and ((data['flow_duration'] > 542.38) or (data['flow_duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Srate': data['Srate'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] > 2.58) or (data['Rate'] == -1)) and ((data['Tot_size'] <= 2061.0) or (data['Tot_size'] == -1)) and ((data['Rate'] <= 250.37) or (data['Rate'] == -1)) and ((data['HTTPS'] > 0.5) or (data['HTTPS'] == -1)) and ((data['flow_duration'] > 136.71) or (data['flow_duration'] == -1)) and ((data['Rate'] > 13.2) or (data['Rate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'Tot_size': data['Tot_size'], 'HTTPS': data['HTTPS'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Threat', 87.88])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 20.15) or (data['Magnitue'] == -1)) and ((data['Srate'] <= 63.78) or (data['Srate'] == -1)) and ((data['flow_duration'] <= 198.58) or (data['flow_duration'] == -1)) and ((data['Srate'] > 5.2) or (data['Srate'] == -1)) and ((data['IAT'] <= 166551312.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 521.15) or (data['Header_Length'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Srate': data['Srate'], 'flow_duration': data['flow_duration'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length']}
        potential_paths.append([feature_dict, 'Threat', 69.7])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 20.15) or (data['Magnitue'] == -1)) and ((data['Covariance'] <= 334993.23) or (data['Covariance'] == -1)) and ((data['Srate'] > 51.52) or (data['Srate'] == -1)) and ((data['Radius'] > 514.89) or (data['Radius'] == -1)) and ((data['Radius'] > 516.97) or (data['Radius'] == -1)) and ((data['Weight'] > 193.08) or (data['Weight'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Covariance': data['Covariance'], 'Srate': data['Srate'], 'Radius': data['Radius'], 'Weight': data['Weight']}
        potential_paths.append([feature_dict, 'Threat', 93.75])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 20.15) or (data['Magnitue'] == -1)) and ((data['Srate'] > 63.78) or (data['Srate'] == -1)) and ((data['flow_duration'] <= 44.61) or (data['flow_duration'] == -1)) and ((data['syn_count'] <= 0.75) or (data['syn_count'] == -1)) and ((data['Rate'] <= 69912.46) or (data['Rate'] == -1)) and ((data['Protocol_Type'] <= 12.55) or (data['Protocol_Type'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Srate': data['Srate'], 'flow_duration': data['flow_duration'], 'syn_count': data['syn_count'], 'Rate': data['Rate'], 'Protocol_Type': data['Protocol_Type']}
        potential_paths.append([feature_dict, 'Threat', 65.52])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] > 193.08) or (data['Weight'] == -1)) and ((data['fin_count'] <= 0.65) or (data['fin_count'] == -1)) and ((data['AVG'] <= 137.6) or (data['AVG'] == -1)) and ((data['Rate'] > 0.21) or (data['Rate'] == -1)) and ((data['Max'] > 224.2) or (data['Max'] == -1)) and ((data['Tot_size'] <= 85.95) or (data['Tot_size'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'fin_count': data['fin_count'], 'AVG': data['AVG'], 'Rate': data['Rate'], 'Max': data['Max'], 'Tot_size': data['Tot_size']}
        potential_paths.append([feature_dict, 'Benign', 89.29])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] > 20.68) or (data['Std'] == -1)) and ((data['ack_flag_number'] <= 0.5) or (data['ack_flag_number'] == -1)) and ((data['urg_count'] > 0.17) or (data['urg_count'] == -1)) and ((data['AVG'] <= 83.0) or (data['AVG'] == -1)) and ((data['AVG'] <= 79.77) or (data['AVG'] == -1)) and ((data['Tot_sum'] <= 1219.45) or (data['Tot_sum'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'ack_flag_number': data['ack_flag_number'], 'AVG': data['AVG']}
        potential_paths.append([feature_dict, 'Threat', 74.07])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 20.15) or (data['Magnitue'] == -1)) and ((data['Srate'] > 63.78) or (data['Srate'] == -1)) and ((data['flow_duration'] <= 44.61) or (data['flow_duration'] == -1)) and ((data['syn_count'] <= 0.75) or (data['syn_count'] == -1)) and ((data['Rate'] <= 69912.46) or (data['Rate'] == -1)) and ((data['Protocol_Type'] > 12.55) or (data['Protocol_Type'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Srate': data['Srate'], 'flow_duration': data['flow_duration'], 'syn_count': data['syn_count'], 'Rate': data['Rate'], 'Protocol_Type': data['Protocol_Type']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] <= 193.08) or (data['Weight'] == -1)) and ((data['IAT'] > 0.02) or (data['IAT'] == -1)) and ((data['Tot_size'] <= 119.09) or (data['Tot_size'] == -1)) and ((data['Variance'] <= 0.89) or (data['Variance'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'Tot_size': data['Tot_size'], 'Variance': data['Variance']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] <= 193.08) or (data['Weight'] == -1)) and ((data['IAT'] <= 0.02) or (data['IAT'] == -1)) and ((data['SSH'] > 0.5) or (data['SSH'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'SSH': data['SSH']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] <= 4.15) or (data['urg_count'] == -1)) and ((data['Duration'] > 127.4) or (data['Duration'] == -1)) and ((data['rst_count'] <= 319.6) or (data['rst_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Duration': data['Duration']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] > 4.15) or (data['urg_count'] == -1)) and ((data['Max'] > 1495.0) or (data['Max'] == -1)) and ((data['Tot_sum'] > 6064.9) or (data['Tot_sum'] == -1)) and ((data['Covariance'] > 1139705.38) or (data['Covariance'] == -1)) and ((data['Tot_sum'] > 7686.1) or (data['Tot_sum'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Max': data['Max'], 'Tot_sum': data['Tot_sum'], 'Covariance': data['Covariance']}
        potential_paths.append([feature_dict, 'Threat', 88.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] <= 193.08) or (data['Weight'] == -1)) and ((data['IAT'] > 0.02) or (data['IAT'] == -1)) and ((data['Tot_size'] > 119.09) or (data['Tot_size'] == -1)) and ((data['fin_count'] <= 0.0) or (data['fin_count'] == -1)) and ((data['Weight'] > 90.03) or (data['Weight'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'Tot_size': data['Tot_size'], 'fin_count': data['fin_count']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] > 2.58) or (data['Rate'] == -1)) and ((data['Tot_size'] > 2061.0) or (data['Tot_size'] == -1)) and ((data['Header_Length'] <= 4577017.75) or (data['Header_Length'] == -1)) and ((data['Variance'] <= 0.95) or (data['Variance'] == -1)) and ((data['flow_duration'] <= 8.23) or (data['flow_duration'] == -1)) and ((data['urg_count'] <= 20.25) or (data['urg_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'Tot_size': data['Tot_size'], 'Variance': data['Variance'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Benign', 90.91])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] <= 193.08) or (data['Weight'] == -1)) and ((data['IAT'] > 0.02) or (data['IAT'] == -1)) and ((data['Tot_size'] <= 119.09) or (data['Tot_size'] == -1)) and ((data['Variance'] > 0.89) or (data['Variance'] == -1)) and ((data['flow_duration'] <= 64.74) or (data['flow_duration'] == -1)) and ((data['Covariance'] <= 3569.78) or (data['Covariance'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'Tot_size': data['Tot_size'], 'Variance': data['Variance'], 'flow_duration': data['flow_duration'], 'Covariance': data['Covariance']}
        potential_paths.append([feature_dict, 'Benign', 50.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 20.15) or (data['Magnitue'] == -1)) and ((data['Covariance'] <= 334993.23) or (data['Covariance'] == -1)) and ((data['Srate'] > 51.52) or (data['Srate'] == -1)) and ((data['Radius'] > 514.89) or (data['Radius'] == -1)) and ((data['Radius'] > 516.97) or (data['Radius'] == -1)) and ((data['Weight'] <= 193.08) or (data['Weight'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Covariance': data['Covariance'], 'Srate': data['Srate'], 'Radius': data['Radius'], 'Weight': data['Weight']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] > 2.58) or (data['Rate'] == -1)) and ((data['Tot_size'] <= 2061.0) or (data['Tot_size'] == -1)) and ((data['Rate'] <= 250.37) or (data['Rate'] == -1)) and ((data['HTTPS'] > 0.5) or (data['HTTPS'] == -1)) and ((data['flow_duration'] > 136.71) or (data['flow_duration'] == -1)) and ((data['Rate'] <= 13.2) or (data['Rate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'Tot_size': data['Tot_size'], 'HTTPS': data['HTTPS'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Threat', 54.55])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] <= 4.15) or (data['urg_count'] == -1)) and ((data['Duration'] <= 127.4) or (data['Duration'] == -1)) and ((data['Tot_sum'] > 3211.7) or (data['Tot_sum'] == -1)) and ((data['IAT'] > 166563728.0) or (data['IAT'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Duration': data['Duration'], 'Tot_sum': data['Tot_sum'], 'IAT': data['IAT']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] <= 436.1) or (data['Tot_sum'] == -1)) and ((data['fin_count'] <= 0.05) or (data['fin_count'] == -1)) and ((data['syn_flag_number'] > 0.5) or (data['syn_flag_number'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'fin_count': data['fin_count'], 'syn_flag_number': data['syn_flag_number']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] <= 193.08) or (data['Weight'] == -1)) and ((data['IAT'] <= 0.02) or (data['IAT'] == -1)) and ((data['SSH'] <= 0.5) or (data['SSH'] == -1)) and ((data['rst_count'] > 77.55) or (data['rst_count'] == -1)) and ((data['Protocol_Type'] <= 6.45) or (data['Protocol_Type'] == -1)) and ((data['flow_duration'] > 57.31) or (data['flow_duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'SSH': data['SSH'], 'rst_count': data['rst_count'], 'Protocol_Type': data['Protocol_Type'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Threat', 95.24])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 20.15) or (data['Magnitue'] == -1)) and ((data['Covariance'] <= 334993.23) or (data['Covariance'] == -1)) and ((data['Srate'] > 51.52) or (data['Srate'] == -1)) and ((data['Radius'] <= 514.89) or (data['Radius'] == -1)) and ((data['ack_flag_number'] > 0.5) or (data['ack_flag_number'] == -1)) and ((data['urg_count'] > 1.05) or (data['urg_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Covariance': data['Covariance'], 'Srate': data['Srate'], 'Radius': data['Radius'], 'ack_flag_number': data['ack_flag_number']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] <= 2.58) or (data['Rate'] == -1)) and ((data['flow_duration'] > 175.81) or (data['flow_duration'] == -1)) and ((data['Magnitue'] > 13.71) or (data['Magnitue'] == -1)) and ((data['Magnitue'] <= 17.55) or (data['Magnitue'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'flow_duration': data['flow_duration'], 'Magnitue': data['Magnitue']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] > 2.58) or (data['Rate'] == -1)) and ((data['Tot_size'] <= 2061.0) or (data['Tot_size'] == -1)) and ((data['Rate'] <= 250.37) or (data['Rate'] == -1)) and ((data['HTTPS'] <= 0.5) or (data['HTTPS'] == -1)) and ((data['IAT'] <= 166499168.0) or (data['IAT'] == -1)) and ((data['Rate'] > 82.11) or (data['Rate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'Tot_size': data['Tot_size'], 'HTTPS': data['HTTPS']}
        potential_paths.append([feature_dict, 'Threat', 68.42])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] > 20.68) or (data['Std'] == -1)) and ((data['ack_flag_number'] <= 0.5) or (data['ack_flag_number'] == -1)) and ((data['urg_count'] > 0.17) or (data['urg_count'] == -1)) and ((data['AVG'] > 83.0) or (data['AVG'] == -1)) and ((data['syn_count'] <= 1.25) or (data['syn_count'] == -1)) and ((data['Max'] <= 167.8) or (data['Max'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'ack_flag_number': data['ack_flag_number'], 'AVG': data['AVG'], 'syn_count': data['syn_count'], 'Max': data['Max']}
        potential_paths.append([feature_dict, 'Threat', 77.78])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 20.15) or (data['Magnitue'] == -1)) and ((data['Covariance'] <= 334993.23) or (data['Covariance'] == -1)) and ((data['Srate'] <= 51.52) or (data['Srate'] == -1)) and ((data['rst_count'] <= 1.35) or (data['rst_count'] == -1)) and ((data['AVG'] > 230.36) or (data['AVG'] == -1)) and ((data['Covariance'] > 100546.71) or (data['Covariance'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Covariance': data['Covariance'], 'Srate': data['Srate'], 'AVG': data['AVG']}
        potential_paths.append([feature_dict, 'Threat', 94.44])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] > 20.68) or (data['Std'] == -1)) and ((data['ack_flag_number'] > 0.5) or (data['ack_flag_number'] == -1)) and ((data['flow_duration'] <= 0.21) or (data['flow_duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'ack_flag_number': data['ack_flag_number'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] <= 4.15) or (data['urg_count'] == -1)) and ((data['Duration'] <= 127.4) or (data['Duration'] == -1)) and ((data['Tot_sum'] <= 3211.7) or (data['Tot_sum'] == -1)) and ((data['flow_duration'] > 1.75) or (data['flow_duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Duration': data['Duration'], 'Tot_sum': data['Tot_sum'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 55.49) or (data['Magnitue'] == -1)) and ((data['Tot_sum'] <= 24485.7) or (data['Tot_sum'] == -1)) and ((data['urg_count'] > 0.4) or (data['urg_count'] == -1)) and ((data['Header_Length'] > 29479.55) or (data['Header_Length'] == -1)) and ((data['flow_duration'] <= 49.51) or (data['flow_duration'] == -1)) and ((data['Max'] > 3418.6) or (data['Max'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Tot_sum': data['Tot_sum'], 'Header_Length': data['Header_Length'], 'flow_duration': data['flow_duration'], 'Max': data['Max']}
        potential_paths.append([feature_dict, 'Benign', 81.25])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 20.15) or (data['Magnitue'] == -1)) and ((data['Covariance'] > 334993.23) or (data['Covariance'] == -1)) and ((data['IAT'] <= 166551360.0) or (data['IAT'] == -1)) and ((data['Protocol_Type'] > 5.92) or (data['Protocol_Type'] == -1)) and ((data['flow_duration'] <= 81.93) or (data['flow_duration'] == -1)) and ((data['Protocol_Type'] > 9.6) or (data['Protocol_Type'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Covariance': data['Covariance'], 'IAT': data['IAT'], 'Protocol_Type': data['Protocol_Type'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Benign', 87.5])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] <= 20.68) or (data['Std'] == -1)) and ((data['Header_Length'] > 51990.2) or (data['Header_Length'] == -1)) and ((data['Header_Length'] > 82806.7) or (data['Header_Length'] == -1)) and ((data['flow_duration'] <= 57.88) or (data['flow_duration'] == -1)) and ((data['Tot_size'] > 99.6) or (data['Tot_size'] == -1)) and ((data['Magnitue'] > 54.84) or (data['Magnitue'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'Header_Length': data['Header_Length'], 'flow_duration': data['flow_duration'], 'Tot_size': data['Tot_size'], 'Magnitue': data['Magnitue']}
        potential_paths.append([feature_dict, 'Threat', 93.33])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] > 20.68) or (data['Std'] == -1)) and ((data['ack_flag_number'] <= 0.5) or (data['ack_flag_number'] == -1)) and ((data['urg_count'] > 0.17) or (data['urg_count'] == -1)) and ((data['AVG'] > 83.0) or (data['AVG'] == -1)) and ((data['syn_count'] <= 1.25) or (data['syn_count'] == -1)) and ((data['Max'] > 167.8) or (data['Max'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'ack_flag_number': data['ack_flag_number'], 'AVG': data['AVG'], 'syn_count': data['syn_count'], 'Max': data['Max']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 20.15) or (data['Magnitue'] == -1)) and ((data['Srate'] <= 63.78) or (data['Srate'] == -1)) and ((data['flow_duration'] <= 198.58) or (data['flow_duration'] == -1)) and ((data['Srate'] <= 5.2) or (data['Srate'] == -1)) and ((data['Duration'] > 60.9) or (data['Duration'] == -1)) and ((data['rst_count'] <= 0.05) or (data['rst_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Srate': data['Srate'], 'flow_duration': data['flow_duration'], 'Duration': data['Duration']}
        potential_paths.append([feature_dict, 'Benign', 50.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] <= 436.1) or (data['Tot_sum'] == -1)) and ((data['fin_count'] <= 0.05) or (data['fin_count'] == -1)) and ((data['syn_flag_number'] <= 0.5) or (data['syn_flag_number'] == -1)) and ((data['Tot_size'] > 57.55) or (data['Tot_size'] == -1)) and ((data['rst_count'] <= 14.15) or (data['rst_count'] == -1)) and ((data['syn_count'] > 1.3) or (data['syn_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'fin_count': data['fin_count'], 'syn_flag_number': data['syn_flag_number'], 'Tot_size': data['Tot_size'], 'rst_count': data['rst_count'], 'syn_count': data['syn_count']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] <= 20.68) or (data['Std'] == -1)) and ((data['Header_Length'] <= 51990.2) or (data['Header_Length'] == -1)) and ((data['urg_count'] <= 2.7) or (data['urg_count'] == -1)) and ((data['Covariance'] > 65.16) or (data['Covariance'] == -1)) and ((data['Protocol_Type'] <= 15.18) or (data['Protocol_Type'] == -1)) and ((data['Header_Length'] > 539.35) or (data['Header_Length'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'Header_Length': data['Header_Length'], 'Covariance': data['Covariance'], 'Protocol_Type': data['Protocol_Type']}
        potential_paths.append([feature_dict, 'Threat', 85.71])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] <= 436.1) or (data['Tot_sum'] == -1)) and ((data['fin_count'] <= 0.05) or (data['fin_count'] == -1)) and ((data['syn_flag_number'] <= 0.5) or (data['syn_flag_number'] == -1)) and ((data['Tot_size'] > 57.55) or (data['Tot_size'] == -1)) and ((data['rst_count'] > 14.15) or (data['rst_count'] == -1)) and ((data['flow_duration'] > 0.84) or (data['flow_duration'] == -1)) and ((data['Std'] > 6.89) or (data['Std'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'fin_count': data['fin_count'], 'syn_flag_number': data['syn_flag_number'], 'Tot_size': data['Tot_size'], 'rst_count': data['rst_count'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Threat', 57.14])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] > 193.08) or (data['Weight'] == -1)) and ((data['fin_count'] <= 0.65) or (data['fin_count'] == -1)) and ((data['AVG'] > 137.6) or (data['AVG'] == -1)) and ((data['IAT'] <= 166499176.0) or (data['IAT'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'fin_count': data['fin_count'], 'AVG': data['AVG']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] > 20.68) or (data['Std'] == -1)) and ((data['ack_flag_number'] <= 0.5) or (data['ack_flag_number'] == -1)) and ((data['urg_count'] <= 0.17) or (data['urg_count'] == -1)) and ((data['Tot_size'] > 55.15) or (data['Tot_size'] == -1)) and ((data['IAT'] <= 41463032.02) or (data['IAT'] == -1)) and ((data['IAT'] <= 0.03) or (data['IAT'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'ack_flag_number': data['ack_flag_number'], 'Tot_size': data['Tot_size'], 'IAT': data['IAT']}
        potential_paths.append([feature_dict, 'Threat', 92.31])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 55.49) or (data['Magnitue'] == -1)) and ((data['Tot_sum'] <= 24485.7) or (data['Tot_sum'] == -1)) and ((data['urg_count'] > 0.4) or (data['urg_count'] == -1)) and ((data['Header_Length'] > 29479.55) or (data['Header_Length'] == -1)) and ((data['flow_duration'] <= 49.51) or (data['flow_duration'] == -1)) and ((data['Max'] <= 3418.6) or (data['Max'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Tot_sum': data['Tot_sum'], 'Header_Length': data['Header_Length'], 'flow_duration': data['flow_duration'], 'Max': data['Max']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] <= 193.08) or (data['Weight'] == -1)) and ((data['IAT'] > 0.02) or (data['IAT'] == -1)) and ((data['Tot_size'] > 119.09) or (data['Tot_size'] == -1)) and ((data['fin_count'] > 0.0) or (data['fin_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'Tot_size': data['Tot_size'], 'fin_count': data['fin_count']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] <= 2.58) or (data['Rate'] == -1)) and ((data['flow_duration'] > 175.81) or (data['flow_duration'] == -1)) and ((data['Magnitue'] <= 13.71) or (data['Magnitue'] == -1)) and ((data['Header_Length'] > 47664.7) or (data['Header_Length'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'flow_duration': data['flow_duration'], 'Magnitue': data['Magnitue']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 55.49) or (data['Magnitue'] == -1)) and ((data['Tot_sum'] > 24485.7) or (data['Tot_sum'] == -1)) and ((data['flow_duration'] > 2.26) or (data['flow_duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Tot_sum': data['Tot_sum'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] <= 436.1) or (data['Tot_sum'] == -1)) and ((data['fin_count'] <= 0.05) or (data['fin_count'] == -1)) and ((data['syn_flag_number'] <= 0.5) or (data['syn_flag_number'] == -1)) and ((data['Tot_size'] > 57.55) or (data['Tot_size'] == -1)) and ((data['rst_count'] <= 14.15) or (data['rst_count'] == -1)) and ((data['syn_count'] <= 1.3) or (data['syn_count'] == -1)) and ((data['Tot_size'] > 81.15) or (data['Tot_size'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'fin_count': data['fin_count'], 'syn_flag_number': data['syn_flag_number'], 'Tot_size': data['Tot_size'], 'rst_count': data['rst_count'], 'syn_count': data['syn_count']}
        potential_paths.append([feature_dict, 'Benign', 63.64])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] > 4.15) or (data['urg_count'] == -1)) and ((data['Max'] <= 1495.0) or (data['Max'] == -1)) and ((data['Srate'] <= 32.65) or (data['Srate'] == -1)) and ((data['IAT'] <= 166561832.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 76886.9) or (data['Header_Length'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Max': data['Max'], 'Srate': data['Srate'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length']}
        potential_paths.append([feature_dict, 'Benign', 54.55])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] <= 436.1) or (data['Tot_sum'] == -1)) and ((data['fin_count'] <= 0.05) or (data['fin_count'] == -1)) and ((data['syn_flag_number'] <= 0.5) or (data['syn_flag_number'] == -1)) and ((data['Tot_size'] > 57.55) or (data['Tot_size'] == -1)) and ((data['rst_count'] > 14.15) or (data['rst_count'] == -1)) and ((data['flow_duration'] <= 0.84) or (data['flow_duration'] == -1)) and ((data['Header_Length'] <= 50053.4) or (data['Header_Length'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'fin_count': data['fin_count'], 'syn_flag_number': data['syn_flag_number'], 'Tot_size': data['Tot_size'], 'rst_count': data['rst_count'], 'flow_duration': data['flow_duration'], 'Header_Length': data['Header_Length']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] > 4.15) or (data['urg_count'] == -1)) and ((data['Max'] <= 1495.0) or (data['Max'] == -1)) and ((data['Srate'] > 32.65) or (data['Srate'] == -1)) and ((data['Header_Length'] > 148793.2) or (data['Header_Length'] == -1)) and ((data['urg_count'] <= 7.55) or (data['urg_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Max': data['Max'], 'Srate': data['Srate'], 'Header_Length': data['Header_Length']}
        potential_paths.append([feature_dict, 'Benign', 80.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 20.15) or (data['Magnitue'] == -1)) and ((data['Srate'] <= 63.78) or (data['Srate'] == -1)) and ((data['flow_duration'] > 198.58) or (data['flow_duration'] == -1)) and ((data['flow_duration'] <= 542.38) or (data['flow_duration'] == -1)) and ((data['Magnitue'] > 13.02) or (data['Magnitue'] == -1)) and ((data['Tot_sum'] <= 619.15) or (data['Tot_sum'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Srate': data['Srate'], 'flow_duration': data['flow_duration'], 'Tot_sum': data['Tot_sum']}
        potential_paths.append([feature_dict, 'Threat', 70.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] <= 436.1) or (data['Tot_sum'] == -1)) and ((data['fin_count'] <= 0.05) or (data['fin_count'] == -1)) and ((data['syn_flag_number'] <= 0.5) or (data['syn_flag_number'] == -1)) and ((data['Tot_size'] > 57.55) or (data['Tot_size'] == -1)) and ((data['rst_count'] > 14.15) or (data['rst_count'] == -1)) and ((data['flow_duration'] > 0.84) or (data['flow_duration'] == -1)) and ((data['Std'] <= 6.89) or (data['Std'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'fin_count': data['fin_count'], 'syn_flag_number': data['syn_flag_number'], 'Tot_size': data['Tot_size'], 'rst_count': data['rst_count'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Threat', 90.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] > 20.68) or (data['Std'] == -1)) and ((data['ack_flag_number'] > 0.5) or (data['ack_flag_number'] == -1)) and ((data['flow_duration'] > 0.21) or (data['flow_duration'] == -1)) and ((data['Srate'] <= 37.11) or (data['Srate'] == -1)) and ((data['Tot_size'] > 84.6) or (data['Tot_size'] == -1)) and ((data['urg_count'] <= 5.85) or (data['urg_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'ack_flag_number': data['ack_flag_number'], 'flow_duration': data['flow_duration'], 'Srate': data['Srate'], 'Tot_size': data['Tot_size']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] <= 20.68) or (data['Std'] == -1)) and ((data['Header_Length'] <= 51990.2) or (data['Header_Length'] == -1)) and ((data['urg_count'] > 2.7) or (data['urg_count'] == -1)) and ((data['syn_count'] > 0.15) or (data['syn_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'Header_Length': data['Header_Length'], 'syn_count': data['syn_count']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] > 4.15) or (data['urg_count'] == -1)) and ((data['Max'] > 1495.0) or (data['Max'] == -1)) and ((data['Tot_sum'] <= 6064.9) or (data['Tot_sum'] == -1)) and ((data['Covariance'] <= 708371.25) or (data['Covariance'] == -1)) and ((data['flow_duration'] <= 4.47) or (data['flow_duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Max': data['Max'], 'Tot_sum': data['Tot_sum'], 'Covariance': data['Covariance'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Benign', 66.67])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] > 20.68) or (data['Std'] == -1)) and ((data['ack_flag_number'] > 0.5) or (data['ack_flag_number'] == -1)) and ((data['flow_duration'] > 0.21) or (data['flow_duration'] == -1)) and ((data['Srate'] > 37.11) or (data['Srate'] == -1)) and ((data['rst_count'] > 15.65) or (data['rst_count'] == -1)) and ((data['Srate'] > 40.56) or (data['Srate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'ack_flag_number': data['ack_flag_number'], 'flow_duration': data['flow_duration'], 'Srate': data['Srate'], 'rst_count': data['rst_count']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] > 20.68) or (data['Std'] == -1)) and ((data['ack_flag_number'] > 0.5) or (data['ack_flag_number'] == -1)) and ((data['flow_duration'] > 0.21) or (data['flow_duration'] == -1)) and ((data['Srate'] > 37.11) or (data['Srate'] == -1)) and ((data['rst_count'] <= 15.65) or (data['rst_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'ack_flag_number': data['ack_flag_number'], 'flow_duration': data['flow_duration'], 'Srate': data['Srate'], 'rst_count': data['rst_count']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] > 4.15) or (data['urg_count'] == -1)) and ((data['Max'] <= 1495.0) or (data['Max'] == -1)) and ((data['Srate'] <= 32.65) or (data['Srate'] == -1)) and ((data['IAT'] <= 166561832.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 76886.9) or (data['Header_Length'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Max': data['Max'], 'Srate': data['Srate'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] <= 193.08) or (data['Weight'] == -1)) and ((data['IAT'] > 0.02) or (data['IAT'] == -1)) and ((data['Tot_size'] <= 119.09) or (data['Tot_size'] == -1)) and ((data['Variance'] > 0.89) or (data['Variance'] == -1)) and ((data['flow_duration'] <= 64.74) or (data['flow_duration'] == -1)) and ((data['Covariance'] > 3569.78) or (data['Covariance'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'Tot_size': data['Tot_size'], 'Variance': data['Variance'], 'flow_duration': data['flow_duration'], 'Covariance': data['Covariance']}
        potential_paths.append([feature_dict, 'Threat', 87.5])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] > 4.15) or (data['urg_count'] == -1)) and ((data['Max'] <= 1495.0) or (data['Max'] == -1)) and ((data['Srate'] > 32.65) or (data['Srate'] == -1)) and ((data['Header_Length'] <= 148793.2) or (data['Header_Length'] == -1)) and ((data['Header_Length'] <= 30030.7) or (data['Header_Length'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Max': data['Max'], 'Srate': data['Srate'], 'Header_Length': data['Header_Length']}
        potential_paths.append([feature_dict, 'Benign', 50.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] > 2.58) or (data['Rate'] == -1)) and ((data['Tot_size'] > 2061.0) or (data['Tot_size'] == -1)) and ((data['Header_Length'] > 4577017.75) or (data['Header_Length'] == -1)) and ((data['Number'] <= 9.5) or (data['Number'] == -1)) and ((data['rst_count'] <= 6670.2) or (data['rst_count'] == -1)) and ((data['Rate'] > 666.97) or (data['Rate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'Tot_size': data['Tot_size'], 'Number': data['Number'], 'rst_count': data['rst_count']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 20.15) or (data['Magnitue'] == -1)) and ((data['Srate'] > 63.78) or (data['Srate'] == -1)) and ((data['flow_duration'] <= 44.61) or (data['flow_duration'] == -1)) and ((data['syn_count'] <= 0.75) or (data['syn_count'] == -1)) and ((data['Rate'] > 69912.46) or (data['Rate'] == -1)) and ((data['rst_count'] > 0.95) or (data['rst_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Srate': data['Srate'], 'flow_duration': data['flow_duration'], 'syn_count': data['syn_count'], 'Rate': data['Rate']}
        potential_paths.append([feature_dict, 'Benign', 75.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] <= 2.58) or (data['Rate'] == -1)) and ((data['flow_duration'] > 175.81) or (data['flow_duration'] == -1)) and ((data['Magnitue'] > 13.71) or (data['Magnitue'] == -1)) and ((data['Magnitue'] > 17.55) or (data['Magnitue'] == -1)) and ((data['Tot_sum'] > 794.65) or (data['Tot_sum'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'flow_duration': data['flow_duration'], 'Magnitue': data['Magnitue'], 'Tot_sum': data['Tot_sum']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] <= 20.68) or (data['Std'] == -1)) and ((data['Header_Length'] > 51990.2) or (data['Header_Length'] == -1)) and ((data['Header_Length'] > 82806.7) or (data['Header_Length'] == -1)) and ((data['flow_duration'] <= 57.88) or (data['flow_duration'] == -1)) and ((data['Tot_size'] <= 99.6) or (data['Tot_size'] == -1)) and ((data['Duration'] <= 103.05) or (data['Duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'Header_Length': data['Header_Length'], 'flow_duration': data['flow_duration'], 'Tot_size': data['Tot_size'], 'Duration': data['Duration']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] > 4.15) or (data['urg_count'] == -1)) and ((data['Max'] > 1495.0) or (data['Max'] == -1)) and ((data['Tot_sum'] <= 6064.9) or (data['Tot_sum'] == -1)) and ((data['Covariance'] <= 708371.25) or (data['Covariance'] == -1)) and ((data['flow_duration'] > 4.47) or (data['flow_duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Max': data['Max'], 'Tot_sum': data['Tot_sum'], 'Covariance': data['Covariance'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Threat', 85.71])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 20.15) or (data['Magnitue'] == -1)) and ((data['Covariance'] > 334993.23) or (data['Covariance'] == -1)) and ((data['IAT'] <= 166551360.0) or (data['IAT'] == -1)) and ((data['Protocol_Type'] <= 5.92) or (data['Protocol_Type'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Covariance': data['Covariance'], 'IAT': data['IAT'], 'Protocol_Type': data['Protocol_Type']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] <= 436.1) or (data['Tot_sum'] == -1)) and ((data['fin_count'] <= 0.05) or (data['fin_count'] == -1)) and ((data['syn_flag_number'] <= 0.5) or (data['syn_flag_number'] == -1)) and ((data['Tot_size'] <= 57.55) or (data['Tot_size'] == -1)) and ((data['Radius'] > 5.94) or (data['Radius'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'fin_count': data['fin_count'], 'syn_flag_number': data['syn_flag_number'], 'Tot_size': data['Tot_size'], 'Radius': data['Radius']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] > 4.15) or (data['urg_count'] == -1)) and ((data['Max'] > 1495.0) or (data['Max'] == -1)) and ((data['Tot_sum'] <= 6064.9) or (data['Tot_sum'] == -1)) and ((data['Covariance'] > 708371.25) or (data['Covariance'] == -1)) and ((data['flow_duration'] > 1.03) or (data['flow_duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Max': data['Max'], 'Tot_sum': data['Tot_sum'], 'Covariance': data['Covariance'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Benign', 85.71])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] <= 436.1) or (data['Tot_sum'] == -1)) and ((data['fin_count'] > 0.05) or (data['fin_count'] == -1)) and ((data['Protocol_Type'] > 10.65) or (data['Protocol_Type'] == -1)) and ((data['fin_count'] <= 0.65) or (data['fin_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'fin_count': data['fin_count'], 'Protocol_Type': data['Protocol_Type']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 20.15) or (data['Magnitue'] == -1)) and ((data['Covariance'] > 334993.23) or (data['Covariance'] == -1)) and ((data['IAT'] <= 166551360.0) or (data['IAT'] == -1)) and ((data['Protocol_Type'] > 5.92) or (data['Protocol_Type'] == -1)) and ((data['flow_duration'] > 81.93) or (data['flow_duration'] == -1)) and ((data['Variance'] <= 0.95) or (data['Variance'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Covariance': data['Covariance'], 'IAT': data['IAT'], 'Protocol_Type': data['Protocol_Type'], 'flow_duration': data['flow_duration'], 'Variance': data['Variance']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] <= 436.1) or (data['Tot_sum'] == -1)) and ((data['fin_count'] <= 0.05) or (data['fin_count'] == -1)) and ((data['syn_flag_number'] <= 0.5) or (data['syn_flag_number'] == -1)) and ((data['Tot_size'] > 57.55) or (data['Tot_size'] == -1)) and ((data['rst_count'] > 14.15) or (data['rst_count'] == -1)) and ((data['flow_duration'] <= 0.84) or (data['flow_duration'] == -1)) and ((data['Header_Length'] > 50053.4) or (data['Header_Length'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'fin_count': data['fin_count'], 'syn_flag_number': data['syn_flag_number'], 'Tot_size': data['Tot_size'], 'rst_count': data['rst_count'], 'flow_duration': data['flow_duration'], 'Header_Length': data['Header_Length']}
        potential_paths.append([feature_dict, 'Benign', 50.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] <= 436.1) or (data['Tot_sum'] == -1)) and ((data['fin_count'] <= 0.05) or (data['fin_count'] == -1)) and ((data['syn_flag_number'] <= 0.5) or (data['syn_flag_number'] == -1)) and ((data['Tot_size'] <= 57.55) or (data['Tot_size'] == -1)) and ((data['Radius'] <= 5.94) or (data['Radius'] == -1)) and ((data['Radius'] <= 5.63) or (data['Radius'] == -1)) and ((data['Rate'] > 3.41) or (data['Rate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'fin_count': data['fin_count'], 'syn_flag_number': data['syn_flag_number'], 'Tot_size': data['Tot_size'], 'Radius': data['Radius'], 'Rate': data['Rate']}
        potential_paths.append([feature_dict, 'Benign', 83.33])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 20.15) or (data['Magnitue'] == -1)) and ((data['Srate'] <= 63.78) or (data['Srate'] == -1)) and ((data['flow_duration'] <= 198.58) or (data['flow_duration'] == -1)) and ((data['Srate'] <= 5.2) or (data['Srate'] == -1)) and ((data['Duration'] <= 60.9) or (data['Duration'] == -1)) and ((data['Srate'] <= 2.94) or (data['Srate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Srate': data['Srate'], 'flow_duration': data['flow_duration'], 'Duration': data['Duration']}
        potential_paths.append([feature_dict, 'Benign', 50.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 55.49) or (data['Magnitue'] == -1)) and ((data['Tot_sum'] > 24485.7) or (data['Tot_sum'] == -1)) and ((data['flow_duration'] <= 2.26) or (data['flow_duration'] == -1)) and ((data['flow_duration'] <= 0.84) or (data['flow_duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Tot_sum': data['Tot_sum'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] > 20.68) or (data['Std'] == -1)) and ((data['ack_flag_number'] > 0.5) or (data['ack_flag_number'] == -1)) and ((data['flow_duration'] > 0.21) or (data['flow_duration'] == -1)) and ((data['Srate'] <= 37.11) or (data['Srate'] == -1)) and ((data['Tot_size'] > 84.6) or (data['Tot_size'] == -1)) and ((data['urg_count'] > 5.85) or (data['urg_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'ack_flag_number': data['ack_flag_number'], 'flow_duration': data['flow_duration'], 'Srate': data['Srate'], 'Tot_size': data['Tot_size']}
        potential_paths.append([feature_dict, 'Threat', 83.33])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] > 20.68) or (data['Std'] == -1)) and ((data['ack_flag_number'] <= 0.5) or (data['ack_flag_number'] == -1)) and ((data['urg_count'] > 0.17) or (data['urg_count'] == -1)) and ((data['AVG'] <= 83.0) or (data['AVG'] == -1)) and ((data['AVG'] > 79.77) or (data['AVG'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'ack_flag_number': data['ack_flag_number'], 'AVG': data['AVG']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 20.15) or (data['Magnitue'] == -1)) and ((data['Covariance'] > 334993.23) or (data['Covariance'] == -1)) and ((data['IAT'] <= 166551360.0) or (data['IAT'] == -1)) and ((data['Protocol_Type'] > 5.92) or (data['Protocol_Type'] == -1)) and ((data['flow_duration'] > 81.93) or (data['flow_duration'] == -1)) and ((data['Variance'] > 0.95) or (data['Variance'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Covariance': data['Covariance'], 'IAT': data['IAT'], 'Protocol_Type': data['Protocol_Type'], 'flow_duration': data['flow_duration'], 'Variance': data['Variance']}
        potential_paths.append([feature_dict, 'Threat', 83.33])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] > 2.58) or (data['Rate'] == -1)) and ((data['Tot_size'] > 2061.0) or (data['Tot_size'] == -1)) and ((data['Header_Length'] > 4577017.75) or (data['Header_Length'] == -1)) and ((data['Number'] > 9.5) or (data['Number'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'Tot_size': data['Tot_size'], 'Number': data['Number']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] > 4.15) or (data['urg_count'] == -1)) and ((data['Max'] > 1495.0) or (data['Max'] == -1)) and ((data['Tot_sum'] > 6064.9) or (data['Tot_sum'] == -1)) and ((data['Covariance'] <= 1139705.38) or (data['Covariance'] == -1)) and ((data['rst_count'] > 474.5) or (data['rst_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Max': data['Max'], 'Tot_sum': data['Tot_sum'], 'Covariance': data['Covariance']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] > 4.15) or (data['urg_count'] == -1)) and ((data['Max'] > 1495.0) or (data['Max'] == -1)) and ((data['Tot_sum'] <= 6064.9) or (data['Tot_sum'] == -1)) and ((data['Covariance'] > 708371.25) or (data['Covariance'] == -1)) and ((data['flow_duration'] <= 1.03) or (data['flow_duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Max': data['Max'], 'Tot_sum': data['Tot_sum'], 'Covariance': data['Covariance'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] > 2.58) or (data['Rate'] == -1)) and ((data['Tot_size'] > 2061.0) or (data['Tot_size'] == -1)) and ((data['Header_Length'] <= 4577017.75) or (data['Header_Length'] == -1)) and ((data['Variance'] <= 0.95) or (data['Variance'] == -1)) and ((data['flow_duration'] > 8.23) or (data['flow_duration'] == -1)) and ((data['Max'] <= 2744.8) or (data['Max'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'Tot_size': data['Tot_size'], 'Variance': data['Variance'], 'flow_duration': data['flow_duration'], 'Max': data['Max']}
        potential_paths.append([feature_dict, 'Benign', 60.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] > 4.15) or (data['urg_count'] == -1)) and ((data['Max'] <= 1495.0) or (data['Max'] == -1)) and ((data['Srate'] > 32.65) or (data['Srate'] == -1)) and ((data['Header_Length'] > 148793.2) or (data['Header_Length'] == -1)) and ((data['urg_count'] > 7.55) or (data['urg_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Max': data['Max'], 'Srate': data['Srate'], 'Header_Length': data['Header_Length']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] > 2.58) or (data['Rate'] == -1)) and ((data['Tot_size'] > 2061.0) or (data['Tot_size'] == -1)) and ((data['Header_Length'] > 4577017.75) or (data['Header_Length'] == -1)) and ((data['Number'] <= 9.5) or (data['Number'] == -1)) and ((data['rst_count'] <= 6670.2) or (data['rst_count'] == -1)) and ((data['Rate'] <= 666.97) or (data['Rate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'Tot_size': data['Tot_size'], 'Number': data['Number'], 'rst_count': data['rst_count']}
        potential_paths.append([feature_dict, 'Threat', 80.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] > 20.68) or (data['Std'] == -1)) and ((data['ack_flag_number'] > 0.5) or (data['ack_flag_number'] == -1)) and ((data['flow_duration'] > 0.21) or (data['flow_duration'] == -1)) and ((data['Srate'] <= 37.11) or (data['Srate'] == -1)) and ((data['Tot_size'] <= 84.6) or (data['Tot_size'] == -1)) and ((data['Protocol_Type'] <= 7.35) or (data['Protocol_Type'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'ack_flag_number': data['ack_flag_number'], 'flow_duration': data['flow_duration'], 'Srate': data['Srate'], 'Tot_size': data['Tot_size'], 'Protocol_Type': data['Protocol_Type']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 20.15) or (data['Magnitue'] == -1)) and ((data['Srate'] <= 63.78) or (data['Srate'] == -1)) and ((data['flow_duration'] > 198.58) or (data['flow_duration'] == -1)) and ((data['flow_duration'] <= 542.38) or (data['flow_duration'] == -1)) and ((data['Magnitue'] <= 13.02) or (data['Magnitue'] == -1)) and ((data['Rate'] <= 3.46) or (data['Rate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Srate': data['Srate'], 'flow_duration': data['flow_duration'], 'Rate': data['Rate']}
        potential_paths.append([feature_dict, 'Threat', 80.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 20.15) or (data['Magnitue'] == -1)) and ((data['Srate'] <= 63.78) or (data['Srate'] == -1)) and ((data['flow_duration'] > 198.58) or (data['flow_duration'] == -1)) and ((data['flow_duration'] <= 542.38) or (data['flow_duration'] == -1)) and ((data['Magnitue'] <= 13.02) or (data['Magnitue'] == -1)) and ((data['Rate'] > 3.46) or (data['Rate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Srate': data['Srate'], 'flow_duration': data['flow_duration'], 'Rate': data['Rate']}
        potential_paths.append([feature_dict, 'Benign', 60.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] <= 20.68) or (data['Std'] == -1)) and ((data['Header_Length'] > 51990.2) or (data['Header_Length'] == -1)) and ((data['Header_Length'] <= 82806.7) or (data['Header_Length'] == -1)) and ((data['Number'] > 11.5) or (data['Number'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'Header_Length': data['Header_Length'], 'Number': data['Number']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] <= 20.68) or (data['Std'] == -1)) and ((data['Header_Length'] > 51990.2) or (data['Header_Length'] == -1)) and ((data['Header_Length'] > 82806.7) or (data['Header_Length'] == -1)) and ((data['flow_duration'] <= 57.88) or (data['flow_duration'] == -1)) and ((data['Tot_size'] <= 99.6) or (data['Tot_size'] == -1)) and ((data['Duration'] > 103.05) or (data['Duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'Header_Length': data['Header_Length'], 'flow_duration': data['flow_duration'], 'Tot_size': data['Tot_size'], 'Duration': data['Duration']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] > 4.15) or (data['urg_count'] == -1)) and ((data['Max'] <= 1495.0) or (data['Max'] == -1)) and ((data['Srate'] <= 32.65) or (data['Srate'] == -1)) and ((data['IAT'] > 166561832.0) or (data['IAT'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Max': data['Max'], 'Srate': data['Srate'], 'IAT': data['IAT']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 20.15) or (data['Magnitue'] == -1)) and ((data['Srate'] <= 63.78) or (data['Srate'] == -1)) and ((data['flow_duration'] <= 198.58) or (data['flow_duration'] == -1)) and ((data['Srate'] <= 5.2) or (data['Srate'] == -1)) and ((data['Duration'] <= 60.9) or (data['Duration'] == -1)) and ((data['Srate'] > 2.94) or (data['Srate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Srate': data['Srate'], 'flow_duration': data['flow_duration'], 'Duration': data['Duration']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] <= 4.15) or (data['urg_count'] == -1)) and ((data['Duration'] <= 127.4) or (data['Duration'] == -1)) and ((data['Tot_sum'] <= 3211.7) or (data['Tot_sum'] == -1)) and ((data['flow_duration'] <= 1.75) or (data['flow_duration'] == -1)) and ((data['Header_Length'] <= 321769.55) or (data['Header_Length'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Duration': data['Duration'], 'Tot_sum': data['Tot_sum'], 'flow_duration': data['flow_duration'], 'Header_Length': data['Header_Length']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] <= 20.68) or (data['Std'] == -1)) and ((data['Header_Length'] <= 51990.2) or (data['Header_Length'] == -1)) and ((data['urg_count'] > 2.7) or (data['urg_count'] == -1)) and ((data['syn_count'] <= 0.15) or (data['syn_count'] == -1)) and ((data['Srate'] > 0.33) or (data['Srate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'Header_Length': data['Header_Length'], 'syn_count': data['syn_count'], 'Srate': data['Srate']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] > 20.68) or (data['Std'] == -1)) and ((data['ack_flag_number'] > 0.5) or (data['ack_flag_number'] == -1)) and ((data['flow_duration'] > 0.21) or (data['flow_duration'] == -1)) and ((data['Srate'] <= 37.11) or (data['Srate'] == -1)) and ((data['Tot_size'] <= 84.6) or (data['Tot_size'] == -1)) and ((data['Protocol_Type'] > 7.35) or (data['Protocol_Type'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'ack_flag_number': data['ack_flag_number'], 'flow_duration': data['flow_duration'], 'Srate': data['Srate'], 'Tot_size': data['Tot_size'], 'Protocol_Type': data['Protocol_Type']}
        potential_paths.append([feature_dict, 'Benign', 66.67])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] <= 4.15) or (data['urg_count'] == -1)) and ((data['Duration'] <= 127.4) or (data['Duration'] == -1)) and ((data['Tot_sum'] <= 3211.7) or (data['Tot_sum'] == -1)) and ((data['flow_duration'] <= 1.75) or (data['flow_duration'] == -1)) and ((data['Header_Length'] > 321769.55) or (data['Header_Length'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Duration': data['Duration'], 'Tot_sum': data['Tot_sum'], 'flow_duration': data['flow_duration'], 'Header_Length': data['Header_Length']}
        potential_paths.append([feature_dict, 'Threat', 66.67])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] > 2.58) or (data['Rate'] == -1)) and ((data['Tot_size'] <= 2061.0) or (data['Tot_size'] == -1)) and ((data['Rate'] > 250.37) or (data['Rate'] == -1)) and ((data['Number'] <= 11.5) or (data['Number'] == -1)) and ((data['rst_count'] > 2541.7) or (data['rst_count'] == -1)) and ((data['flow_duration'] > 75.49) or (data['flow_duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'Tot_size': data['Tot_size'], 'Number': data['Number'], 'rst_count': data['rst_count'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Benign', 66.67])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] > 193.08) or (data['Weight'] == -1)) and ((data['fin_count'] > 0.65) or (data['fin_count'] == -1)) and ((data['Magnitue'] <= 32.27) or (data['Magnitue'] == -1)) and ((data['Duration'] <= 106.25) or (data['Duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'fin_count': data['fin_count'], 'Magnitue': data['Magnitue'], 'Duration': data['Duration']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] > 193.08) or (data['Weight'] == -1)) and ((data['fin_count'] <= 0.65) or (data['fin_count'] == -1)) and ((data['AVG'] <= 137.6) or (data['AVG'] == -1)) and ((data['Rate'] > 0.21) or (data['Rate'] == -1)) and ((data['Max'] <= 224.2) or (data['Max'] == -1)) and ((data['Std'] <= 24.99) or (data['Std'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'fin_count': data['fin_count'], 'AVG': data['AVG'], 'Rate': data['Rate'], 'Max': data['Max'], 'Std': data['Std']}
        potential_paths.append([feature_dict, 'Threat', 66.67])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] > 193.08) or (data['Weight'] == -1)) and ((data['fin_count'] <= 0.65) or (data['fin_count'] == -1)) and ((data['AVG'] <= 137.6) or (data['AVG'] == -1)) and ((data['Rate'] <= 0.21) or (data['Rate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'fin_count': data['fin_count'], 'AVG': data['AVG'], 'Rate': data['Rate']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 55.49) or (data['Magnitue'] == -1)) and ((data['Tot_sum'] > 24485.7) or (data['Tot_sum'] == -1)) and ((data['flow_duration'] <= 2.26) or (data['flow_duration'] == -1)) and ((data['flow_duration'] > 0.84) or (data['flow_duration'] == -1)) and ((data['urg_count'] > 1.0) or (data['urg_count'] == -1)) and ((data['AVG'] > 2165.93) or (data['AVG'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Tot_sum': data['Tot_sum'], 'flow_duration': data['flow_duration'], 'AVG': data['AVG']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 20.15) or (data['Magnitue'] == -1)) and ((data['Srate'] > 63.78) or (data['Srate'] == -1)) and ((data['flow_duration'] <= 44.61) or (data['flow_duration'] == -1)) and ((data['syn_count'] <= 0.75) or (data['syn_count'] == -1)) and ((data['Rate'] > 69912.46) or (data['Rate'] == -1)) and ((data['rst_count'] <= 0.95) or (data['rst_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Srate': data['Srate'], 'flow_duration': data['flow_duration'], 'syn_count': data['syn_count'], 'Rate': data['Rate']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 55.49) or (data['Magnitue'] == -1)) and ((data['Tot_sum'] <= 24485.7) or (data['Tot_sum'] == -1)) and ((data['urg_count'] <= 0.4) or (data['urg_count'] == -1)) and ((data['Magnitue'] > 61.2) or (data['Magnitue'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Tot_sum': data['Tot_sum']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] > 4.15) or (data['urg_count'] == -1)) and ((data['Max'] > 1495.0) or (data['Max'] == -1)) and ((data['Tot_sum'] > 6064.9) or (data['Tot_sum'] == -1)) and ((data['Covariance'] > 1139705.38) or (data['Covariance'] == -1)) and ((data['Tot_sum'] <= 7686.1) or (data['Tot_sum'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Max': data['Max'], 'Tot_sum': data['Tot_sum'], 'Covariance': data['Covariance']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] > 20.68) or (data['Std'] == -1)) and ((data['ack_flag_number'] > 0.5) or (data['ack_flag_number'] == -1)) and ((data['flow_duration'] > 0.21) or (data['flow_duration'] == -1)) and ((data['Srate'] > 37.11) or (data['Srate'] == -1)) and ((data['rst_count'] > 15.65) or (data['rst_count'] == -1)) and ((data['Srate'] <= 40.56) or (data['Srate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'ack_flag_number': data['ack_flag_number'], 'flow_duration': data['flow_duration'], 'Srate': data['Srate'], 'rst_count': data['rst_count']}
        potential_paths.append([feature_dict, 'Benign', 50.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] <= 20.68) or (data['Std'] == -1)) and ((data['Header_Length'] > 51990.2) or (data['Header_Length'] == -1)) and ((data['Header_Length'] > 82806.7) or (data['Header_Length'] == -1)) and ((data['flow_duration'] > 57.88) or (data['flow_duration'] == -1)) and ((data['Tot_size'] <= 58.8) or (data['Tot_size'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'Header_Length': data['Header_Length'], 'flow_duration': data['flow_duration'], 'Tot_size': data['Tot_size']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] > 2.58) or (data['Rate'] == -1)) and ((data['Tot_size'] > 2061.0) or (data['Tot_size'] == -1)) and ((data['Header_Length'] > 4577017.75) or (data['Header_Length'] == -1)) and ((data['Number'] <= 9.5) or (data['Number'] == -1)) and ((data['rst_count'] > 6670.2) or (data['rst_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'Tot_size': data['Tot_size'], 'Number': data['Number'], 'rst_count': data['rst_count']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] <= 2.58) or (data['Rate'] == -1)) and ((data['flow_duration'] > 175.81) or (data['flow_duration'] == -1)) and ((data['Magnitue'] <= 13.71) or (data['Magnitue'] == -1)) and ((data['Header_Length'] <= 47664.7) or (data['Header_Length'] == -1)) and ((data['Radius'] > 24.08) or (data['Radius'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'flow_duration': data['flow_duration'], 'Magnitue': data['Magnitue'], 'Radius': data['Radius']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] > 193.08) or (data['Weight'] == -1)) and ((data['fin_count'] > 0.65) or (data['fin_count'] == -1)) and ((data['Magnitue'] <= 32.27) or (data['Magnitue'] == -1)) and ((data['Duration'] > 106.25) or (data['Duration'] == -1)) and ((data['Rate'] > 20.57) or (data['Rate'] == -1)) and ((data['IAT'] <= 166478680.0) or (data['IAT'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'fin_count': data['fin_count'], 'Magnitue': data['Magnitue'], 'Duration': data['Duration'], 'Rate': data['Rate']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 55.49) or (data['Magnitue'] == -1)) and ((data['Tot_sum'] > 24485.7) or (data['Tot_sum'] == -1)) and ((data['flow_duration'] <= 2.26) or (data['flow_duration'] == -1)) and ((data['flow_duration'] > 0.84) or (data['flow_duration'] == -1)) and ((data['urg_count'] > 1.0) or (data['urg_count'] == -1)) and ((data['AVG'] <= 2165.93) or (data['AVG'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Tot_sum': data['Tot_sum'], 'flow_duration': data['flow_duration'], 'AVG': data['AVG']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 55.49) or (data['Magnitue'] == -1)) and ((data['Tot_sum'] <= 24485.7) or (data['Tot_sum'] == -1)) and ((data['urg_count'] > 0.4) or (data['urg_count'] == -1)) and ((data['Header_Length'] > 29479.55) or (data['Header_Length'] == -1)) and ((data['flow_duration'] > 49.51) or (data['flow_duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Tot_sum': data['Tot_sum'], 'Header_Length': data['Header_Length'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] <= 4.15) or (data['urg_count'] == -1)) and ((data['Duration'] <= 127.4) or (data['Duration'] == -1)) and ((data['Tot_sum'] > 3211.7) or (data['Tot_sum'] == -1)) and ((data['IAT'] <= 166563728.0) or (data['IAT'] == -1)) and ((data['AVG'] > 1508.07) or (data['AVG'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Duration': data['Duration'], 'Tot_sum': data['Tot_sum'], 'IAT': data['IAT'], 'AVG': data['AVG']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] <= 20.68) or (data['Std'] == -1)) and ((data['Header_Length'] > 51990.2) or (data['Header_Length'] == -1)) and ((data['Header_Length'] <= 82806.7) or (data['Header_Length'] == -1)) and ((data['Number'] <= 11.5) or (data['Number'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'Header_Length': data['Header_Length'], 'Number': data['Number']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 20.15) or (data['Magnitue'] == -1)) and ((data['Covariance'] <= 334993.23) or (data['Covariance'] == -1)) and ((data['Srate'] > 51.52) or (data['Srate'] == -1)) and ((data['Radius'] <= 514.89) or (data['Radius'] == -1)) and ((data['ack_flag_number'] > 0.5) or (data['ack_flag_number'] == -1)) and ((data['urg_count'] <= 1.05) or (data['urg_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Covariance': data['Covariance'], 'Srate': data['Srate'], 'Radius': data['Radius'], 'ack_flag_number': data['ack_flag_number']}
        potential_paths.append([feature_dict, 'Benign', 50.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] > 20.68) or (data['Std'] == -1)) and ((data['ack_flag_number'] <= 0.5) or (data['ack_flag_number'] == -1)) and ((data['urg_count'] > 0.17) or (data['urg_count'] == -1)) and ((data['AVG'] <= 83.0) or (data['AVG'] == -1)) and ((data['AVG'] <= 79.77) or (data['AVG'] == -1)) and ((data['Tot_sum'] > 1219.45) or (data['Tot_sum'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'ack_flag_number': data['ack_flag_number'], 'AVG': data['AVG']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 20.15) or (data['Magnitue'] == -1)) and ((data['Covariance'] <= 334993.23) or (data['Covariance'] == -1)) and ((data['Srate'] <= 51.52) or (data['Srate'] == -1)) and ((data['rst_count'] > 1.35) or (data['rst_count'] == -1)) and ((data['Rate'] <= 50.38) or (data['Rate'] == -1)) and ((data['Duration'] <= 48.35) or (data['Duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Covariance': data['Covariance'], 'Srate': data['Srate'], 'Rate': data['Rate'], 'Duration': data['Duration']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] <= 20.68) or (data['Std'] == -1)) and ((data['Header_Length'] > 51990.2) or (data['Header_Length'] == -1)) and ((data['Header_Length'] > 82806.7) or (data['Header_Length'] == -1)) and ((data['flow_duration'] > 57.88) or (data['flow_duration'] == -1)) and ((data['Tot_size'] > 58.8) or (data['Tot_size'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'Header_Length': data['Header_Length'], 'flow_duration': data['flow_duration'], 'Tot_size': data['Tot_size']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 20.15) or (data['Magnitue'] == -1)) and ((data['Covariance'] <= 334993.23) or (data['Covariance'] == -1)) and ((data['Srate'] <= 51.52) or (data['Srate'] == -1)) and ((data['rst_count'] <= 1.35) or (data['rst_count'] == -1)) and ((data['AVG'] <= 230.36) or (data['AVG'] == -1)) and ((data['flow_duration'] <= 128.99) or (data['flow_duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Covariance': data['Covariance'], 'Srate': data['Srate'], 'AVG': data['AVG'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 20.15) or (data['Magnitue'] == -1)) and ((data['Srate'] > 63.78) or (data['Srate'] == -1)) and ((data['flow_duration'] <= 44.61) or (data['flow_duration'] == -1)) and ((data['syn_count'] > 0.75) or (data['syn_count'] == -1)) and ((data['TCP'] <= 0.5) or (data['TCP'] == -1)) and ((data['IAT'] <= 83819576.01) or (data['IAT'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Srate': data['Srate'], 'flow_duration': data['flow_duration'], 'syn_count': data['syn_count'], 'TCP': data['TCP'], 'IAT': data['IAT']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] <= 436.1) or (data['Tot_sum'] == -1)) and ((data['fin_count'] <= 0.05) or (data['fin_count'] == -1)) and ((data['syn_flag_number'] <= 0.5) or (data['syn_flag_number'] == -1)) and ((data['Tot_size'] <= 57.55) or (data['Tot_size'] == -1)) and ((data['Radius'] <= 5.94) or (data['Radius'] == -1)) and ((data['Radius'] > 5.63) or (data['Radius'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'fin_count': data['fin_count'], 'syn_flag_number': data['syn_flag_number'], 'Tot_size': data['Tot_size'], 'Radius': data['Radius']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 20.15) or (data['Magnitue'] == -1)) and ((data['Srate'] > 63.78) or (data['Srate'] == -1)) and ((data['flow_duration'] <= 44.61) or (data['flow_duration'] == -1)) and ((data['syn_count'] > 0.75) or (data['syn_count'] == -1)) and ((data['TCP'] <= 0.5) or (data['TCP'] == -1)) and ((data['IAT'] > 83819576.01) or (data['IAT'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Srate': data['Srate'], 'flow_duration': data['flow_duration'], 'syn_count': data['syn_count'], 'TCP': data['TCP'], 'IAT': data['IAT']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] <= 2.58) or (data['Rate'] == -1)) and ((data['flow_duration'] > 175.81) or (data['flow_duration'] == -1)) and ((data['Magnitue'] > 13.71) or (data['Magnitue'] == -1)) and ((data['Magnitue'] > 17.55) or (data['Magnitue'] == -1)) and ((data['Tot_sum'] <= 794.65) or (data['Tot_sum'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'flow_duration': data['flow_duration'], 'Magnitue': data['Magnitue'], 'Tot_sum': data['Tot_sum']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 20.15) or (data['Magnitue'] == -1)) and ((data['Srate'] > 63.78) or (data['Srate'] == -1)) and ((data['flow_duration'] <= 44.61) or (data['flow_duration'] == -1)) and ((data['syn_count'] > 0.75) or (data['syn_count'] == -1)) and ((data['TCP'] > 0.5) or (data['TCP'] == -1)) and ((data['Max'] <= 179.8) or (data['Max'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Srate': data['Srate'], 'flow_duration': data['flow_duration'], 'syn_count': data['syn_count'], 'TCP': data['TCP'], 'Max': data['Max']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] <= 2.58) or (data['Rate'] == -1)) and ((data['flow_duration'] > 175.81) or (data['flow_duration'] == -1)) and ((data['Magnitue'] <= 13.71) or (data['Magnitue'] == -1)) and ((data['Header_Length'] <= 47664.7) or (data['Header_Length'] == -1)) and ((data['Radius'] <= 24.08) or (data['Radius'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'flow_duration': data['flow_duration'], 'Magnitue': data['Magnitue'], 'Radius': data['Radius']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] > 41682.35) or (data['Header_Length'] == -1)) and ((data['Rate'] <= 2.58) or (data['Rate'] == -1)) and ((data['flow_duration'] <= 175.81) or (data['flow_duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Rate': data['Rate'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] > 193.08) or (data['Weight'] == -1)) and ((data['fin_count'] > 0.65) or (data['fin_count'] == -1)) and ((data['Magnitue'] > 32.27) or (data['Magnitue'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'fin_count': data['fin_count'], 'Magnitue': data['Magnitue']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] > 193.08) or (data['Weight'] == -1)) and ((data['fin_count'] > 0.65) or (data['fin_count'] == -1)) and ((data['Magnitue'] <= 32.27) or (data['Magnitue'] == -1)) and ((data['Duration'] > 106.25) or (data['Duration'] == -1)) and ((data['Rate'] > 20.57) or (data['Rate'] == -1)) and ((data['IAT'] > 166478680.0) or (data['IAT'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'fin_count': data['fin_count'], 'Magnitue': data['Magnitue'], 'Duration': data['Duration'], 'Rate': data['Rate']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] > 20.68) or (data['Std'] == -1)) and ((data['ack_flag_number'] <= 0.5) or (data['ack_flag_number'] == -1)) and ((data['urg_count'] > 0.17) or (data['urg_count'] == -1)) and ((data['AVG'] > 83.0) or (data['AVG'] == -1)) and ((data['syn_count'] > 1.25) or (data['syn_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'ack_flag_number': data['ack_flag_number'], 'AVG': data['AVG'], 'syn_count': data['syn_count']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] > 8.35) or (data['urg_count'] == -1)) and ((data['IAT'] <= 166551400.0) or (data['IAT'] == -1)) and ((data['Header_Length'] <= 41682.35) or (data['Header_Length'] == -1)) and ((data['Weight'] > 193.08) or (data['Weight'] == -1)) and ((data['fin_count'] > 0.65) or (data['fin_count'] == -1)) and ((data['Magnitue'] <= 32.27) or (data['Magnitue'] == -1)) and ((data['Duration'] > 106.25) or (data['Duration'] == -1)) and ((data['Rate'] <= 20.57) or (data['Rate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'IAT': data['IAT'], 'Header_Length': data['Header_Length'], 'Weight': data['Weight'], 'fin_count': data['fin_count'], 'Magnitue': data['Magnitue'], 'Duration': data['Duration'], 'Rate': data['Rate']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] <= 436.1) or (data['Tot_sum'] == -1)) and ((data['fin_count'] > 0.05) or (data['fin_count'] == -1)) and ((data['Protocol_Type'] > 10.65) or (data['Protocol_Type'] == -1)) and ((data['fin_count'] > 0.65) or (data['fin_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'fin_count': data['fin_count'], 'Protocol_Type': data['Protocol_Type']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 20.15) or (data['Magnitue'] == -1)) and ((data['Covariance'] <= 334993.23) or (data['Covariance'] == -1)) and ((data['Srate'] <= 51.52) or (data['Srate'] == -1)) and ((data['rst_count'] <= 1.35) or (data['rst_count'] == -1)) and ((data['AVG'] <= 230.36) or (data['AVG'] == -1)) and ((data['flow_duration'] > 128.99) or (data['flow_duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Covariance': data['Covariance'], 'Srate': data['Srate'], 'AVG': data['AVG'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] > 20.68) or (data['Std'] == -1)) and ((data['ack_flag_number'] <= 0.5) or (data['ack_flag_number'] == -1)) and ((data['urg_count'] <= 0.17) or (data['urg_count'] == -1)) and ((data['Tot_size'] <= 55.15) or (data['Tot_size'] == -1)) and ((data['flow_duration'] <= 60.13) or (data['flow_duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'ack_flag_number': data['ack_flag_number'], 'Tot_size': data['Tot_size'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 20.15) or (data['Magnitue'] == -1)) and ((data['Covariance'] <= 334993.23) or (data['Covariance'] == -1)) and ((data['Srate'] <= 51.52) or (data['Srate'] == -1)) and ((data['rst_count'] > 1.35) or (data['rst_count'] == -1)) and ((data['Rate'] > 50.38) or (data['Rate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Covariance': data['Covariance'], 'Srate': data['Srate'], 'Rate': data['Rate']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 55.49) or (data['Magnitue'] == -1)) and ((data['Tot_sum'] > 24485.7) or (data['Tot_sum'] == -1)) and ((data['flow_duration'] <= 2.26) or (data['flow_duration'] == -1)) and ((data['flow_duration'] > 0.84) or (data['flow_duration'] == -1)) and ((data['urg_count'] <= 1.0) or (data['urg_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Tot_sum': data['Tot_sum'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] <= 20.68) or (data['Std'] == -1)) and ((data['Header_Length'] <= 51990.2) or (data['Header_Length'] == -1)) and ((data['urg_count'] > 2.7) or (data['urg_count'] == -1)) and ((data['syn_count'] <= 0.15) or (data['syn_count'] == -1)) and ((data['Srate'] <= 0.33) or (data['Srate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'Header_Length': data['Header_Length'], 'syn_count': data['syn_count'], 'Srate': data['Srate']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 55.49) or (data['Magnitue'] == -1)) and ((data['Tot_sum'] <= 24485.7) or (data['Tot_sum'] == -1)) and ((data['urg_count'] > 0.4) or (data['urg_count'] == -1)) and ((data['Header_Length'] <= 29479.55) or (data['Header_Length'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Tot_sum': data['Tot_sum'], 'Header_Length': data['Header_Length']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] > 20.68) or (data['Std'] == -1)) and ((data['ack_flag_number'] <= 0.5) or (data['ack_flag_number'] == -1)) and ((data['urg_count'] <= 0.17) or (data['urg_count'] == -1)) and ((data['Tot_size'] > 55.15) or (data['Tot_size'] == -1)) and ((data['IAT'] <= 41463032.02) or (data['IAT'] == -1)) and ((data['IAT'] > 0.03) or (data['IAT'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'ack_flag_number': data['ack_flag_number'], 'Tot_size': data['Tot_size'], 'IAT': data['IAT']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] <= 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 20.15) or (data['Magnitue'] == -1)) and ((data['Covariance'] <= 334993.23) or (data['Covariance'] == -1)) and ((data['Srate'] > 51.52) or (data['Srate'] == -1)) and ((data['Radius'] > 514.89) or (data['Radius'] == -1)) and ((data['Radius'] <= 516.97) or (data['Radius'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Covariance': data['Covariance'], 'Srate': data['Srate'], 'Radius': data['Radius']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] > 55.49) or (data['Magnitue'] == -1)) and ((data['Tot_sum'] <= 24485.7) or (data['Tot_sum'] == -1)) and ((data['urg_count'] <= 0.4) or (data['urg_count'] == -1)) and ((data['Magnitue'] <= 61.2) or (data['Magnitue'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Tot_sum': data['Tot_sum']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] > 436.1) or (data['Tot_sum'] == -1)) and ((data['Std'] > 20.68) or (data['Std'] == -1)) and ((data['ack_flag_number'] <= 0.5) or (data['ack_flag_number'] == -1)) and ((data['urg_count'] <= 0.17) or (data['urg_count'] == -1)) and ((data['Tot_size'] <= 55.15) or (data['Tot_size'] == -1)) and ((data['flow_duration'] > 60.13) or (data['flow_duration'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'ack_flag_number': data['ack_flag_number'], 'Tot_size': data['Tot_size'], 'flow_duration': data['flow_duration']}
        potential_paths.append([feature_dict, 'Benign', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] > 39.83) or (data['Std'] == -1)) and ((data['rst_count'] > 21.55) or (data['rst_count'] == -1)) and ((data['Magnitue'] <= 55.49) or (data['Magnitue'] == -1)) and ((data['urg_count'] <= 4.15) or (data['urg_count'] == -1)) and ((data['Duration'] > 127.4) or (data['Duration'] == -1)) and ((data['rst_count'] > 319.6) or (data['rst_count'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'rst_count': data['rst_count'], 'Magnitue': data['Magnitue'], 'Duration': data['Duration']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    if ((data['urg_count'] <= 8.35) or (data['urg_count'] == -1)) and ((data['Std'] <= 39.83) or (data['Std'] == -1)) and ((data['Tot_sum'] <= 436.1) or (data['Tot_sum'] == -1)) and ((data['fin_count'] <= 0.05) or (data['fin_count'] == -1)) and ((data['syn_flag_number'] <= 0.5) or (data['syn_flag_number'] == -1)) and ((data['Tot_size'] <= 57.55) or (data['Tot_size'] == -1)) and ((data['Radius'] <= 5.94) or (data['Radius'] == -1)) and ((data['Radius'] <= 5.63) or (data['Radius'] == -1)) and ((data['Rate'] <= 3.41) or (data['Rate'] == -1)):
        feature_dict = {'urg_count': data['urg_count'], 'Std': data['Std'], 'Tot_sum': data['Tot_sum'], 'fin_count': data['fin_count'], 'syn_flag_number': data['syn_flag_number'], 'Tot_size': data['Tot_size'], 'Radius': data['Radius'], 'Rate': data['Rate']}
        potential_paths.append([feature_dict, 'Threat', 100.0])
    all_entries_res.append(potential_paths)
    return None

In [10]:
for index, row in df.iterrows():
    selected_features = random.sample(all_features, 0)
    
    for feature in selected_features:
        df.at[index, feature] = -1
    if df.at[index, "fin_flag_number"] == -1:
        df.at[index, "fin_count"] = -1
    if df.at[index, "syn_flag_number"] == -1:
        df.at[index, "syn_count"] = -1
    if df.at[index, "rst_flag_number"] == -1:
        df.at[index, "rst_count"] = -1
    if df.at[index, "ack_flag_number"] == -1:
        df.at[index, "ack_count"] = -1

In [11]:
df.head()

,flow_duration,Header_Length,Protocol_Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,ack_flag_number,ece_flag_number,cwr_flag_number,ack_count,syn_count,fin_count,urg_count,rst_count,HTTP,HTTPS,DNS,Telnet,SMTP,SSH,IRC,TCP,UDP,DHCP,ARP,ICMP,IPv,LLC,Tot_sum,Min,Max,AVG,Std,Tot_size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,0.007704,54.66,6.0,64.0,24.948199,24.948199,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.99,0.01,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,567.72,54.0,54.72,54.041541,0.167652,54.12,8.298532e+07,9.5,10.396320,0.238004,4.728548e-01,0.06,141.55,Threat
1,83.780723,6082636.00,6.0,231.0,69.198495,69.198495,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.00,0.00,231.0,5796.50,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,363.00,66.0,66.00,66.000000,0.000000,66.00,1.186132e-04,5.5,11.489125,0.000000,0.000000e+00,0.00,38.50,Benign
2,2.716553,216465.70,6.0,80.7,77.573022,77.573022,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.80,0.00,12.0,204.50,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,299.40,54.0,56.40,54.253333,0.737124,55.20,8.392096e-04,5.5,10.416540,1.042450,2.718222e+00,0.20,38.50,Benign
3,8.567097,666035.00,6.0,144.9,173.321221,173.321221,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.40,0.00,56.0,623.90,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,25458.00,367.0,4410.00,1692.497682,1121.462154,515.10,1.665249e+08,13.5,58.116108,1585.665641,1.269694e+06,1.00,244.60,Benign
4,0.000000,54.00,6.0,64.0,2.208773,2.208773,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,567.00,54.0,54.00,54.000000,0.000000,54.00,8.308912e+07,9.5,10.392305,0.000000,0.000000e+00,0.00,141.55,Threat


In [12]:
# df.to_csv('../missing_feature/1_missing/dt_1_1missing_features.csv', index=False)

In [13]:
all_entries_res = []

In [14]:
df.apply(lambda row: classify(row), axis=1)

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
30      None
31      None
32      None
33      None
34      None
35      None
36      None
37      None
38      None
39      None
40      None
41      None
42      None
43      None
44      None
45      None
46      None
47      None
48      None
49      None
50      None
51      None
52      None
53      None
54      None
55      None
56      None
57      None
58      None
59      None
60      None
61      None
62      None
63      None
64      None
65      None
66      None
67      None
68      None
69      None
70      None
71      None
72      None
73      None
74      None
75      None
76      None

In [15]:
count = 1
for sublist in all_entries_res:
    for list in sublist:
        entry = list[0]
        # print(str(entry) + "  " + str(list[1]) + "   " + str(list[2]))
        if all(value == -1 for value in entry.values()):
            print("All values in this dictionary are -1.")
            print(count)
    count +=1
    # print("\n\n")

In [16]:
for sublist in all_entries_res:
    # Check if the sublist has at least two elements
    if len(sublist) >= 2:
        # Check if all lists in the sublist are the same (some cases use the same features set to determine and result are the same, however the threshold values could be different. But
        # it looks use the same paths, but indeed it is not)
        if all(sublist[0] == lst for lst in sublist):
            print("All lists in this sublist are the same:", sublist)

In [17]:
#used when value is zero
feature_descriptions_zero = {
    'ts': "Timestamp",
    'flow_duration': "The Duration of the packet's flow",
    'Header_Length': "Packet header length",
    'Protocol_Type': "Protocol type",
    'Duration': "Time-to-Live",
    'Rate': "Rate of packet transmission",
    'Srate': "Rate of outbound packets transmission",
    'Drate': "Rate of inbound packets transmission",
    'fin_flag_number': "Count of FIN flags (which signal the end of data transmission)",
    'syn_flag_number': "Count of SYN flags (indicating the initiation of a TCP three-way handshake)",
    'rst_flag_number': "Count of RST flags (used for resetting connections)",
    'psh_flag_number': "Count of PSH flags (instructing to push buffered data to the receiving application)",
    'ack_flag_number': "Count of ACK flags (used to acknowledge packet receipts)",
    'ece_flag_number': "Count of ECE flags (indicating network congestion)",
    'cwr_flag_number': "Count of CWR flags (signaling reduced transmission rate due to congestion notification)",
    'ack_count': "Number of packets with an ACK flag",
    'syn_count': "Number of packets with a SYN flag",
    'fin_count': "Number of packets with a FIN flag",
    'urg_count': "Number of packets with an URG flag",
    'rst_count': "Number of packets with an RST flag",
    'HTTP': "Application layer protocol is HTTP",
    'HTTPS': "Application layer protocol is HTTPS",
    'DNS': "Application layer protocol is DNS",
    'Telnet': "Application layer protocol is Telnet",
    'SMTP': "Application layer protocol is SMTP",
    'SSH': "Application layer protocol is SSH",
    'IRC': "Application layer protocol is IRC",
    'TCP': "Transport layer protocol is TCP",
    'UDP': "Transport layer protocol is UDP",
    'DHCP': "Application layer protocol is DHCP",
    'ARP': "Link layer protocol is ARP",
    'ICMP': "Network layer protocol is ICMP",
    'IPv': "Network layer protocol is IP",
    'LLC': "Link layer protocol is LLC",
    'Tot_sum': "Total sum of packets lengths",
    'Min': "Minimum packet length in the flow",
    'Max': "Maximum packet length in the flow",
    'AVG': "Average packet length in the flow",
    'Std': "Standard deviation of packet length",
    'Tot_size': "Packet's length",
    'IAT': "The time difference between two consecutive packets",
    'Number': "The total number of packets",
    'Magnitue': "Magnitude (calculated as the mean of the average lengths of incoming and outgoing packets in the flow)",
    'Radius': "Radius (calculated as the averaged variances of packet lengths for incoming and outgoing packets)",
    'Covariance': "Covariance (coveriance of the lengths of incoming and outgoing packets)",
    'Variance': "Variance (calculated as the ratio of variances in packet lengths between incoming and outgoing packets)",
    'Weight': "Weight (calculated as the product of the counts of incoming and outgoing packets)"
}

feature_descriptions = {
    'ts': "Timestamp",
    'flow_duration': "On a scale from Level 1 to Level 21, The Duration of the packet's flow",
    'Header_Length': "On a scale from Level 1 to Level 15, Packet header length",
    'Protocol_Type': "Protocol type",
    'Duration': "Time-to-Live",
    'Rate': "On a scale from Level 1 to Level 12, Rate of packet transmission",
    'Srate': "On a scale from Level 1 to Level 11, Rate of outbound packets transmission",
    'Drate': "Rate of inbound packets transmission",
    'fin_flag_number': "Count of FIN flags (which signal the end of data transmission)",
    'syn_flag_number': "Count of SYN flags (indicating the initiation of a TCP three-way handshake)",
    'rst_flag_number': "Count of RST flags (used for resetting connections)",
    'psh_flag_number': "Count of PSH flags (instructing to push buffered data to the receiving application)",
    'ack_flag_number': "Count of ACK flags (used to acknowledge packet receipts)",
    'ece_flag_number': "Count of ECE flags (indicating network congestion)",
    'cwr_flag_number': "Count of CWR flags (signaling reduced transmission rate due to congestion notification)",
    'ack_count': "Number of packets with an ACK flag",
    'syn_count': "Number of packets with a SYN flag",
    'fin_count': "Number of packets with a FIN flag",
    'urg_count': "On a scale from Level 1 to Level 11, Number of packets with an URG flag",
    'rst_count': "On a scale from Level 1 to Level 12, Number of packets with an RST flag",
    'HTTP': "Application layer protocol is HTTP",
    'HTTPS': "Application layer protocol is HTTPS",
    'DNS': "Application layer protocol is DNS",
    'Telnet': "Application layer protocol is Telnet",
    'SMTP': "Application layer protocol is SMTP",
    'SSH': "Application layer protocol is SSH",
    'IRC': "Application layer protocol is IRC",
    'TCP': "Transport layer protocol is TCP",
    'UDP': "Transport layer protocol is UDP",
    'DHCP': "Application layer protocol is DHCP",
    'ARP': "Link layer protocol is ARP",
    'ICMP': "Network layer protocol is ICMP",
    'IPv': "Network layer protocol is IP",
    'LLC': "Link layer protocol is LLC",
    'Tot_sum': "On a scale from Level 1 to Level 9, Total sum of packets lengths",
    'Min': "Minimum packet length in the flow",
    'Max': "Maximum packet length in the flow",
    'AVG': "On a scale from Level 1 to Level 8, Average packet length in the flow",
    'Std': "Standard deviation of packet length",
    'Tot_size': "On a scale from Level 1 to Level 10, Packet's length",
    'IAT': "On a scale from Level 1 to Level 13, The time difference between two consecutive packets",
    'Number': "The total number of packets",
    'Magnitue': "On a scale from Level 1 to Level 9, Magnitude (calculated as the mean of the average lengths of incoming and outgoing packets in the flow)",
    'Radius': "Radius (calculated as the averaged variances of packet lengths for incoming and outgoing packets)",
    'Covariance': "Covariance (coveriance of the lengths of incoming and outgoing packets)",
    'Variance': "Variance (calculated as the ratio of variances in packet lengths between incoming and outgoing packets)",
    'Weight': "Weight (calculated as the product of the counts of incoming and outgoing packets)"
}

In [18]:
layer_features_unknown = {
    'HTTP': "Application layer protocol is unknown",
    'HTTPS': "Application layer protocol is unknown",
    'DNS': "Application layer protocol is unknown",
    'Telnet': "Application layer protocol is unknown",
    'SMTP': "Application layer protocol is unknown",
    'SSH': "Application layer protocol is unknown",
    'IRC': "Application layer protocol is unknown",
    'TCP': "Transport layer protocol is unknown",
    'UDP': "Transport layer protocol is unknown",
    'DHCP': "Application layer protocol is unknown",
    'ARP': "Link layer protocol is unknown",
    'ICMP': "Network layer protocol is unknown",
    'IPv': "Network layer protocol is unknown",
    'LLC': "Link layer protocol is unknown",
}

In [19]:
def create_description(row):
    descriptions = []
    # for numerical feature
    for feature in numerical_features:
        value = row.get(feature)
        if pd.notnull(value):  # Check if the feature is not NaN
            if value == "unknown":
                descriptions.append(f"{feature_descriptions_zero.get(feature, feature)} is {value}.")
            elif value == "zero":
                descriptions.append(f"{feature_descriptions_zero.get(feature, feature)} is {value}.")
            else:
                descriptions.append(f"{feature_descriptions.get(feature, feature)} is {value}.")
            
    # flag
    for feature in flag_features:
        if row.get(feature) == -1:
            descriptions.append(f"{layer_features_unknown.get(feature, feature)}.")
        elif row.get(feature) == 1:
            protocol_name = ' '.join(feature.split('_')).capitalize()
            descriptions.append(f"{feature_descriptions.get(feature, feature)}.")
            
    if pd.notnull(row.get("psh_flag_number")):
        value = row.get("psh_flag_number")
        if value == -1:
            descriptions.append("The existance of psh(push) flag is unknown")
        elif value == 1:
            descriptions.append("The psh(push) flag exists in this traffic")
            
    if pd.notnull(row.get("rst_flag_number")):
        value = row.get("rst_flag_number")
        rst_count = row.get("rst_count")
        if value == -1:
            descriptions.append("The existance of rst(reset) flag is unknown")
        elif value == 1 and rst_count != "zero":
            descriptions.append("The rst(reset) flag exists in this traffic")
            
    if pd.notnull(row.get("ack_flag_number")):
        value = row.get("ack_flag_number")
        ack_count = row.get("ack_count")
        if value == -1:
            descriptions.append("The existance of ack(Acknowledgment) flag is unknown")
        elif value == 1 and ack_count != "zero":
            descriptions.append("The ack(Acknowledgment) flag exists in this traffic")
            
    if pd.notnull(row.get("syn_flag_number")):
        value = row.get("syn_flag_number")
        syn_count = row.get("syn_count")
        if value == -1:
            descriptions.append("The existance of syn(Synchronize) flag is unknown")
        elif value == 1 and syn_count != "zero":
            descriptions.append("The syn(Synchronize) flag exists in this traffic")

    return ' '.join(descriptions)

In [20]:
def categorize_urg_count(value):
    if value == 0.0:
        return 'zero'
    elif value <= 0.17:
        return 'level 1'
    elif value <= 0.4:
        return 'level 2'
    elif value <= 1.0:
        return 'level 3'
    elif value <= 1.05:
        return 'level 4'
    elif value <= 2.7:
        return 'level 5'
    elif value <= 4.15:
        return 'level 6'
    elif value <= 5.85:
        return 'level 7'
    elif value <= 7.55:
        return 'level 8'
    elif value <= 8.35:
        return 'level 9'
    elif value <= 20.25:
        return 'level 10'
    else:
        return 'level 11'

def categorize_std(value):
    if value == 0.0:
        return 'zero'
    elif value <= 6.89:
        return 'extremely low'
    elif value <= 20.68:
        return 'low'
    elif value <= 24.99:
        return 'relatively low'
    elif value <= 39.83:
        return 'relatively high'
    elif value <= 87.1:
        return 'high'
    else:
        return 'extremely high'

def categorize_tot_sum(value):
    if value == 0.0:
        return 'zero'
    elif value <= 436.1:
        return 'level 1'
    elif value <= 619.15:
        return 'level 2'
    elif value <= 794.65:
        return 'level 3'
    elif value <= 1219.45:
        return 'level 4'
    elif value <= 3211.7:
        return 'level 5'
    elif value <= 6064.9:
        return 'level 6'
    elif value <= 7686.1:
        return 'level 7'
    elif value <= 24485.7:
        return 'level 8'
    else:
        return 'level 9'

def categorize_fin_count(value):
    if value == 0.0:
        return 'zero'
    elif value <= 0.05:
        return 'low'
    elif value <= 0.65:
        return 'medium'
    else:
        return 'high'

def categorize_tot_size(value):
    if value == 0.0:
        return 'zero'
    elif value <= 55.15:
        return 'level 1'
    elif value <= 57.55:
        return 'level 2'
    elif value <= 58.8:
        return 'level 3'
    elif value <= 81.15:
        return 'level 4'
    elif value <= 84.6:
        return 'level 5'
    elif value <= 85.95:
        return 'level 6'
    elif value <= 99.6:
        return 'level 7'
    elif value <= 119.09:
        return 'level 8'
    elif value <= 2061.0:
        return 'level 9'
    else:
        return 'level 10'

def categorize_radius(value):
    if value == 0.0:
        return 'zero'
    elif value <= 5.63:
        return 'extremely low'
    elif value <= 5.94:
        return 'low'
    elif value <= 24.08:
        return 'relatively low'
    elif value <= 73.61:
        return 'medium'
    elif value <= 514.89:
        return 'relatively high'
    elif value <= 516.97:
        return 'high'
    else:
        return 'extremely high'

def categorize_rate(value):
    if value == 0.0:
        return 'zero'
    elif value <= 0.21:
        return 'level 1'
    elif value <= 2.58:
        return 'level 2'
    elif value <= 3.41:
        return 'level 3'
    elif value <= 3.46:
        return 'level 4'
    elif value <= 13.2:
        return 'level 5'
    elif value <= 20.57:
        return 'level 6'
    elif value <= 50.38:
        return 'level 7'
    elif value <= 82.11:
        return 'level 8'
    elif value <= 250.37:
        return 'level 9'
    elif value <= 666.97:
        return 'level 10'
    elif value <= 69912.46:
        return 'level 11'
    else:
        return 'level 12'

def categorize_rst_count(value):
    if value == 0.0:
        return 'zero'
    elif value <= 0.05:
        return 'level 1'
    elif value <= 0.95:
        return 'level 2'
    elif value <= 1.35:
        return 'level 3'
    elif value <= 14.15:
        return 'level 4'
    elif value <= 15.65:
        return 'level 5'
    elif value <= 21.55:
        return 'level 6'
    elif value <= 77.55:
        return 'level 7'
    elif value <= 319.6:
        return 'level 8'
    elif value <= 474.5:
        return 'level 9'
    elif value <= 2541.7:
        return 'level 10'
    elif value <= 6670.2:
        return 'level 11'
    else:
        return 'level 12'

def categorize_syn_count(value):
    if value == 0.0:
        return 'zero'
    elif value <= 0.15:
        return 'low'
    elif value <= 0.75:
        return 'relatively low'
    elif value <= 1.25:
        return 'medium'
    elif value <= 1.3:
        return 'relatively high'
    else:
        return 'high'

def categorize_flow_duration(value):
    if value == 0.0:
        return 'zero'
    elif value <= 0.21:
        return 'level 1'
    elif value <= 0.84:
        return 'level 2'
    elif value <= 1.03:
        return 'level 3'
    elif value <= 1.75:
        return 'level 4'
    elif value <= 2.26:
        return 'level 5'
    elif value <= 4.47:
        return 'level 6'
    elif value <= 8.23:
        return 'level 7'
    elif value <= 44.61:
        return 'level 8'
    elif value <= 49.51:
        return 'level 9'
    elif value <= 57.31:
        return 'level 10'
    elif value <= 57.88:
        return 'level 11'
    elif value <= 60.13:
        return 'level 12'
    elif value <= 64.74:
        return 'level 13'
    elif value <= 75.49:
        return 'level 14'
    elif value <= 81.93:
        return 'level 15'
    elif value <= 128.99:
        return 'level 16'
    elif value <= 136.71:
        return 'level 17'
    elif value <= 175.81:
        return 'level 18'
    elif value <= 198.58:
        return 'level 19'
    elif value <= 542.38:
        return 'level 20'
    else:
        return 'level 21'

def categorize_header_length(value):
    if value == 0.0:
        return 'zero'
    elif value <= 521.15:
        return 'level 1'
    elif value <= 539.35:
        return 'level 2'
    elif value <= 22829.0:
        return 'level 3'
    elif value <= 29479.55:
        return 'level 4'
    elif value <= 30030.7:
        return 'level 5'
    elif value <= 41682.35:
        return 'level 6'
    elif value <= 47664.7:
        return 'level 7'
    elif value <= 50053.4:
        return 'level 8'
    elif value <= 51990.2:
        return 'level 9'
    elif value <= 76886.9:
        return 'level 10'
    elif value <= 82806.7:
        return 'level 11'
    elif value <= 148793.2:
        return 'level 12'
    elif value <= 321769.55:
        return 'level 13'
    elif value <= 4577017.75:
        return 'level 14'
    else:
        return 'level 15'

# def categorize_protocol_type(value):

def categorize_covariance(value):
    if value == 0.0:
        return 'zero'
    elif value <= 65.16:
        return 'extremely low'
    elif value <= 3569.78:
        return 'low'
    elif value <= 100546.71:
        return 'relatively low'
    elif value <= 334993.23:
        return 'medium'
    elif value <= 708371.25:
        return 'relatively high'
    elif value <= 1139705.38:
        return 'high'
    else:
        return 'extremely high'

def categorize_srate(value):
    if value == 0.0:
        return 'zero'
    elif value <= 0.33:
        return 'level 1'
    elif value <= 2.67:
        return 'level 2'
    elif value <= 2.94:
        return 'level 3'
    elif value <= 5.2:
        return 'level 4'
    elif value <= 32.65:
        return 'level 5'
    elif value <= 37.11:
        return 'level 6'
    elif value <= 40.56:
        return 'level 7'
    elif value <= 51.52:
        return 'level 8'
    elif value <= 63.78:
        return 'level 9'
    elif value <= 2093.32:
        return 'level 10'
    else:
        return 'level 11'

def categorize_number(value):
    if value == 0.0:
        return 'zero'
    elif value <= 9.5:
        return 'low'
    elif value <= 11.5:
        return 'medium'
    else:
        return 'high'

def categorize_duration(value):
    if value == 0.0:
        return 'zero'
    elif value <= 48.35:
        return 'extremely low'
    elif value <= 60.9:
        return 'low'
    elif value <= 103.05:
        return 'relatively low'
    elif value <= 106.25:
        return 'relatively high'
    elif value <= 127.4:
        return 'high'
    else:
        return 'extremely high'

def categorize_magnitude(value):
    if value == 0.0:
        return 'zero'
    elif value <= 13.02:
        return 'level 1'
    elif value <= 13.71:
        return 'level 2'
    elif value <= 17.55:
        return 'level 3'
    elif value <= 20.15:
        return 'level 4'
    elif value <= 32.27:
        return 'level 5'
    elif value <= 54.84:
        return 'level 6'
    elif value <= 55.49:
        return 'level 7'
    elif value <= 61.2:
        return 'level 8'
    else:
        return 'level 9'

# def categorize_ack_flag_number(value):

def categorize_iat(value):
    if value == 0.0:
        return 'zero'
    elif value <= 0.02:
        return 'level 1'
    elif value <= 0.03:
        return 'level 2'
    elif value <= 41463032.02:
        return 'level 3'
    elif value <= 83819576.01:
        return 'level 4'
    elif value <= 166478680.0:
        return 'level 5'
    elif value <= 166499168.0:
        return 'level 6'
    elif value <= 166499176.0:
        return 'level 7'
    elif value <= 166551312.0:
        return 'level 8'
    elif value <= 166551360.0:
        return 'level 9'
    elif value <= 166551400.0:
        return 'level 10'
    elif value <= 166561832.0:
        return 'level 11'
    elif value <= 166563728.0:
        return 'level 12'
    else:
        return 'level 13'

def categorize_avg(value):
    if value == 0.0:
        return 'zero'
    elif value <= 79.77:
        return 'level 1'
    elif value <= 83.0:
        return 'level 2'
    elif value <= 137.6:
        return 'level 3'
    elif value <= 223.55:
        return 'level 4'
    elif value <= 230.36:
        return 'level 5'
    elif value <= 1508.07:
        return 'level 6'
    elif value <= 2165.93:
        return 'level 7'
    else:
        return 'level 8'

def categorize_max(value):
    if value == 0.0:
        return 'zero'
    elif value <= 167.8:
        return 'extremely low'
    elif value <= 179.8:
        return 'low'
    elif value <= 224.2:
        return 'relatively low'
    elif value <= 1495.0:
        return 'medium'
    elif value <= 2744.8:
        return 'relatively high'
    elif value <= 3418.6:
        return 'high'
    else:
        return 'extremely high'

def categorize_weight(value):
    if value == 0.0:
        return 'zero'
    elif value <= 90.03:
        return 'low'
    elif value <= 193.08:
        return 'medium'
    else:
        return 'high'

def categorize_variance(value):
    if value == 0.0:
        return 'zero'
    elif value <= 0.89:
        return 'low'
    elif value <= 0.95:
        return 'relatively low'
    elif value <= 0.97:
        return 'relatively high'
    else:
        return 'high'

def categorize_min(value):
    if value == 0.0:
        return 'zero'
    elif value <= 64.81:
        return 'low'
    else:
        return 'high'

In [21]:
def serialize_prediction_path(dict):
    if 'urg_count' in dict:
        if dict['urg_count'] == -1:
            dict['urg_count'] = "unknown"
        else:
            dict['urg_count'] = categorize_urg_count(dict['urg_count'])
            
    if 'Std' in dict:
        if dict['Std'] == -1:
            dict['Std'] = "unknown"
        else:
            dict['Std'] = categorize_std(dict['Std'])

    if 'Tot_sum' in dict:
        if dict['Tot_sum'] == -1:
            dict['Tot_sum'] = "unknown"
        else:
            dict['Tot_sum'] = categorize_tot_sum(dict['Tot_sum'])

    if 'fin_count' in dict:
        if dict['fin_count'] == -1:
            dict['fin_count'] = "unknown"
        else:
            dict['fin_count'] = categorize_fin_count(dict['fin_count'])

    if 'Tot_size' in dict:
        if dict['Tot_size'] == -1:
            dict['Tot_size'] = "unknown"
        else:
            dict['Tot_size'] = categorize_tot_size(dict['Tot_size'])

    if 'Radius' in dict:
        if dict['Radius'] == -1:
            dict['Radius'] = "unknown"
        else:
            dict['Radius'] = categorize_radius(dict['Radius'])

    if 'Rate' in dict:
        if dict['Rate'] == -1:
            dict['Rate'] = "unknown"
        else:
            dict['Rate'] = categorize_rate(dict['Rate'])

    if 'rst_count' in dict:
        if dict['rst_count'] == -1:
            dict['rst_count'] = "unknown"
        else:
            dict['rst_count'] = categorize_rst_count(dict['rst_count'])

    if 'syn_count' in dict:
        if dict['syn_count'] == -1:
            dict['syn_count'] = "unknown"
        else:
            dict['syn_count'] = categorize_syn_count(dict['syn_count'])

    if 'flow_duration' in dict:
        if dict['flow_duration'] == -1:
            dict['flow_duration'] = "unknown"
        else:
            dict['flow_duration'] = categorize_flow_duration(dict['flow_duration'])

    if 'Header_Length' in dict:
        if dict['Header_Length'] == -1:
            dict['Header_Length'] = "unknown"
        else:
            dict['Header_Length'] = categorize_header_length(dict['Header_Length'])

    if 'Covariance' in dict:
        if dict['Covariance'] == -1:
            dict['Covariance'] = "unknown"
        else:
            dict['Covariance'] = categorize_covariance(dict['Covariance'])

    if 'Srate' in dict:
        if dict['Srate'] == -1:
            dict['Srate'] = "unknown"
        else:
            dict['Srate'] = categorize_srate(dict['Srate'])

    if 'Number' in dict:
        if dict['Number'] == -1:
            dict['Number'] = "unknown"
        else:
            dict['Number'] = categorize_number(dict['Number'])

    if 'Duration' in dict:
        if dict['Duration'] == -1:
            dict['Duration'] = "unknown"
        else:
            dict['Duration'] = categorize_duration(dict['Duration'])

    if 'Magnitue' in dict:
        if dict['Magnitue'] == -1:
            dict['Magnitue'] = "unknown"
        else:
            dict['Magnitue'] = categorize_magnitude(dict['Magnitue'])
    
    if 'IAT' in dict:
        if dict['IAT'] == -1:
            dict['IAT'] = "unknown"
        else:
            dict['IAT'] = categorize_iat(dict['IAT'])
    
    if 'AVG' in dict:
        if dict['AVG'] == -1:
            dict['AVG'] = "unknown"
        else:
            dict['AVG'] = categorize_avg(dict['AVG'])
    
    if 'Max' in dict:
        if dict['Max'] == -1:
            dict['Max'] = "unknown"
        else:
            dict['Max'] = categorize_max(dict['Max'])  

    if 'Weight' in dict:
        if dict['Weight'] == -1:
            dict['Weight'] = "unknown"
        else:
            dict['Weight'] = categorize_weight(dict['Weight'])
    
    if 'Variance' in dict:
        if dict['Variance'] == -1:
            dict['Variance'] = "unknown"
        else:
            dict['Variance'] = categorize_variance(dict['Variance'])
    
    if 'Min' in dict:
        if dict['Min'] == -1:
            dict['Min'] = "unknown"
        else:
            dict['Min'] = categorize_min(dict['Min'])
    
    if 'Protocol_Type' in dict:
        dict.pop('Protocol_Type')
        
    return create_description(dict)

In [22]:
for sublist in all_entries_res:
    for list in sublist:
        description = serialize_prediction_path(list[0])
        list.append(description)
        entry = list[0]
        print(str(entry) + "  " + str(list[1]) + "   " + str(list[2]) + "   " + str(list[3]))
        if all(value == -1 for value in entry.values()):
            print("All values in this dictionary are -1.")
    print("\n\n")

{'urg_count': 'zero', 'Std': 'extremely low', 'Tot_sum': 'level 2', 'Header_Length': 'level 1', 'Covariance': 'extremely low'}  Threat   100.0   On a scale from Level 1 to Level 15, Packet header length is level 1. Number of packets with an URG flag is zero. On a scale from Level 1 to Level 9, Total sum of packets lengths is level 2. Standard deviation of packet length is extremely low. Covariance (coveriance of the lengths of incoming and outgoing packets) is extremely low.



{'urg_count': 'level 11', 'IAT': 'level 1', 'Header_Length': 'level 15', 'Rate': 'level 8', 'Tot_size': 'level 4', 'HTTPS': 1.0, 'flow_duration': 'level 16', 'Variance': 'zero'}  Benign   91.77   On a scale from Level 1 to Level 21, The Duration of the packet's flow is level 16. On a scale from Level 1 to Level 15, Packet header length is level 15. On a scale from Level 1 to Level 12, Rate of packet transmission is level 8. On a scale from Level 1 to Level 11, Number of packets with an URG flag is level 11. On a

In [23]:
final_list = []

In [24]:
len_list = []

In [25]:
idx = 0

In [26]:
for sublist in all_entries_res:
    #correct_label = sublist_1[0][1]
    correct_label = df.iloc[idx]["label"]
    idx += 1
    all_paths = []
    for list in sublist:
        all_unknown = False
        prob = list[2]
        if all(value == 'unknown' for value in list[0].values()):
            all_unknown = True
            print(str(idx) + str(list[0]))
        path = [list[0], list[3], list[1], all_unknown]
        if (all_unknown == False):
            all_paths.append(path)
    final_list.append([correct_label, all_paths, prob])

In [27]:
with open('output/dt_1.json', 'w') as file:
    # Serialize list to JSON with indentation and write it to the file
    json.dump(final_list, file, indent=4)